In [ ]:
import subprocess
%run "{os.path.dirname(os.getcwd())}\general_functions\generalFunctions.ipynb" 

## Price Positioning  Summary 

In [ ]:
def pricepositionSummary(prs, data, numOfDuplication, dfGroup,slideby, position=0):
    """
    Update PowerPoint presentation with Promo Sales per Retailer data.

    Parameters:
    prs (Presentation): PowerPoint presentation object to modify.
    endOfWeek (dict): Dictionary containing end of week data.
    numOfDuplicates (int): Number of slides to duplicate and update.
    dfGroup (list): List containing dataframes grouped for each slide.
    position (int, optional): Starting slide position in the presentation. Defaults to 0.

    Returns:
    Replace the slides with new data
    """
    # Define dictionaries to map chart numbers to chart order
    ch1 = {0: 0}
    ch2 = {0: 0, 1: 1}
    ch3 = {0: 0, 1: 1, 2: 2}
    ch4 = {0: 0, 1:1 , 2: 2, 3: 3}
    # Iterate through each slide to update
    i =0
    updated_dfGroup = []

    for group in dfGroup:
        # If the group has more than 6 rows, split it into chunks of size 6
        if len(group) > 5:
            for i in range(0, len(group),4 ):
                updated_dfGroup.append(group[i:i+4])  # Add chunks to the updated list
        else:
            updated_dfGroup.append(group)  # Add the group as is if it's 6 or fewer rows

    numOfDuplicates = len(updated_dfGroup)
    
    for slide_num in range(numOfDuplicates):
        actual_slide_index = slide_num + numOfDuplication
        dfs = updated_dfGroup[slide_num]        
        shapes = prs.slides[actual_slide_index].shapes
        cat= dfs[0].split(' | ')[0]     
        retailer_ = dfs[0].split(' | ')[1]    
        # shapes = prs.slides[slide_num+ position].shapes

        # # Find and update title shape
        titleNumber = get_shape_number(shapes,'Brand Price & Index vs Market | Bubble Size by Value Sales | Category | National | P12M')
        datasourcenum = get_shape_number(shapes, "DATA SOURCE: Trade Panel/Retailer Data | Ending Sep 2022")
        headerNumber = get_shape_number(shapes,'Price Positioning Summary (Replace with SO WHAT)')
        avgbox = get_shape_number(shapes,"Avg Price / Unit")

        if titleNumber is not None:
            shapes[datasourcenum].text = data_source
            shapes[titleNumber].text = f'Brand Price & Index vs Market | Bubble Size by Value Sales | {slideby} | {retailer_} | P12M'
            shapes[titleNumber].text_frame.paragraphs[0].font.size = Pt(12)
            shapes[titleNumber].text_frame.paragraphs[0].font.name = 'Nexa Bold (Headings)'
            shapes[headerNumber].text_frame.paragraphs[0].font.size = Pt(16)
            shapes[headerNumber].text_frame.paragraphs[0].font.name = 'Nexa Bold (Headings)'

        # Create table and chart objects
        tables, charts = createTableAndChart(shapes)
        # Determine the appropriate chart order dictionary based on the number of charts
        chDic = ch2 if len(charts) == 2 else ch3 if len(charts) == 3 else ch4 if len(charts) == 4 else ch1


        for chartNum in range(min(len(charts), len(dfs))):
            chart = charts[chDic[chartNum]].chart

            chart_data = BubbleChartData()            
            key = dfs[chartNum]
            chart_df = data[key]
            
            chart_data.categories = chart_df['Av Price/Unit'].unique().tolist()
            series = chart_data.add_series("Relative Price Index")
            series.has_data_labels = True
            # Add data points to the bubble chart
            for i in range(chart_df.shape[0]):
                series.add_data_point(chart_df['Av Price/Unit'].iloc[i], chart_df['Relative Price'].iloc[i], chart_df['Value Sales'].iloc[i])
            chart.replace_data(chart_data)
            
            category_axis = chart.category_axis
            category_axis.tick_labels.number_format = '#,##0.00'  if decimals == 2 else '#,##0'
            currencywithoutspace =currency.strip()  # Remove the leading space
            shapes[avgbox].text = f"Avg Price/Unit ({currencywithoutspace})"  # Set the axis title text
            shapes[avgbox].text_frame.paragraphs[0].font.size = Pt(8)
            shapes[avgbox].text_frame.paragraphs[0].font.name = 'Nexa Bold'
            shapes[avgbox].text_frame.paragraphs[0].runs[0].font.color.rgb = RGBColor(87, 85, 85)  # Set text color to white
            shapes[avgbox].text_frame.paragraphs[0].alignment = PP_ALIGN.CENTER
            category_axis.auto_axis = True
            min_value = chart_df['Av Price/Unit'].min()
            max_value = chart_df['Av Price/Unit'].max()

            # Set X-axis min to 80% of min_value and max to 120% of max_value
            category_axis.minimum_scale = round(min_value * 0.8)
            category_axis.maximum_scale =round(max_value * 1.2)
            
            value_axis = chart.value_axis
            value_axis.tick_labels.number_format = '0%'
            value_axis = chart.value_axis
            # value_axis.auto_axis = True
            value_axis.minimum_scale = 0
            
            
            xlsx_file=BytesIO()
            with chart_data._workbook_writer._open_worksheet(xlsx_file) as (workbook, worksheet):
                chart_data._workbook_writer._populate_worksheet(workbook, worksheet)
                worksheet.write(0, 4, f'{BrandOrTopB}')
                worksheet.write_column(1, 4, chart_df[f'{BrandOrTopB}'], None)

            chart._workbook.update_from_xlsx_blob(xlsx_file.getvalue())
        
            table = tables[0].table
            for i, row in enumerate(table.rows):
                for j, cell in enumerate(row.cells):
                    if i < len(dfs):  # Ensure you're within bounds of dfs
                        category = dfs[i].split(' | ')[0]
                        row.cells[i].text = category
                        i=i+1
                        cell.text_frame.paragraphs[0].font.name = 'Nexa Book'
                        cell.text_frame.paragraphs[0].font.size = Pt(11)
                        cell.text_frame.paragraphs[0].runs[0].font.bold = True
                        cell.text_frame.paragraphs[0].runs[0].font.color.rgb = RGBColor(87, 85, 85)  # Set text color to white
                        cell.text_frame.paragraphs[0].alignment = PP_ALIGN.CENTER


# Pricing Positioning

In [ ]:
def pricePositioning(prs,modified_price_positioning_sorted,numOfDuplicates,position=0,slideby=""):
    """
    Generate slides for price positioning analysis and bubble chart visualization.

    Args:
        prs: PowerPoint presentation object.
        modified_price_positioning_sorted: Dictionary containing sorted price positioning dataframes.
        numOfDuplicates: Number of duplicate slides to generate.
        position: Position index to start adding slides (default is 0).
    """

    markets = list(modified_price_positioning_sorted.keys())
    for slidenum in range(min(numOfDuplicates, len(markets))):
        market = markets[slidenum]
        df = modified_price_positioning_sorted[market].reset_index(drop=True).copy()
        rel_price = pd.to_numeric(df['Relative Price'], errors='coerce')

        df = df.fillna({c: 0 for c in df.select_dtypes(include='number').columns})

        slide = prs.slides[slidenum + position]
        shapes = slide.shapes

        headerNumber = get_shape_number(shapes, "Price Positioning Analysis (Replace with SO WHAT)")
        titleNumber = get_shape_number(shapes, "Brand Price & Index vs Market | Bubble Size by Value Sales | Category | National | P12M")
        shapes[titleNumber-1].text = data_source
        shapes[titleNumber].text = (
            f'Brand Price & Index vs Market | Bubble Size by Value Sales | {market} | P12M'
            if slideby == f'{BrandOrTopB}'
            else f'Manufactory Price & Index vs Market | Bubble Size by Value Sales | {market} | P12M'
        )
        shapes[titleNumber].text_frame.paragraphs[0].font.size = Pt(12)
        shapes[titleNumber].text_frame.paragraphs[0].font.name = 'Nexa (Headings)'

        chart_shape = None
        for shape in shapes:
            if getattr(shape, "has_chart", False):
                chart_shape = shape
                break
        if chart_shape is None:
            continue
        chart = chart_shape.chart
        chart_shape.left = Inches(0.57)

        # Build the bubble chart data
        chart_data = BubbleChartData()
        chart_data.categories = df['Av Price/Unit'].unique().tolist()

        series = chart_data.add_series("Relative Price Index") 
        series.has_data_labels = True

        # Add data points to the bubble chart
        for i in range(df.shape[0]):
            series.add_data_point(df['Av Price/Unit'].iloc[i], df['Relative Price'].iloc[i], df['Value Sales'].iloc[i])
        chart.replace_data(chart_data)

        # Update chart formatting
        xlsx_file = BytesIO()
        with chart_data._workbook_writer._open_worksheet(xlsx_file) as (workbook, worksheet):
            chart_data._workbook_writer._populate_worksheet(workbook, worksheet)
            worksheet.write(0, 4, "labels")
            worksheet.write_column(1, 4, df[slideby], None)

        chart._workbook.update_from_xlsx_blob(xlsx_file.getvalue())

        category_axis = chart.category_axis
        category_axis.tick_labels.number_format = '#,##0.00' if decimals == 2 else '#,##0'
        currencywithoutspace = currency.strip()
        category_axis.axis_title.text_frame.text = f"Avg Price/Unit ({currencywithoutspace})"
        category_axis.axis_title.text_frame.paragraphs[0].font.size = Pt(8)
        category_axis.axis_title.text_frame.paragraphs[0].font.name = 'Nexa Bold'
        category_axis.auto_axis = True

        min_value = df['Av Price/Unit'].min()
        max_value = df['Av Price/Unit'].max()
        # Set X-axis min to 80% of min_value and max to 120% of max_value
        category_axis.minimum_scale = round(min_value * 0.8)
        category_axis.maximum_scale =round(max_value * 1.2)

        value_axis = chart.value_axis
        value_axis.tick_labels.number_format = '0%'
        value_axis.minimum_scale = 0

        ymin = value_axis.minimum_scale
        ymax = value_axis.maximum_scale

            
        ymax = float(rel_price.max()+0.2) # add margin above max data point
        value_axis.maximum_scale = ymax

        # Tier constraints and bounds
        tier_constraints = {
            "Low-Tier":        lambda s: (s < 0.79),
            "Mid-Tier":        lambda s: (s >= 0.8) & (s <= 1.19),
            "Premium":         lambda s: (s > 1.2) & (s <= 1.79),
            "Super Premium":   lambda s: (s > 1.8),
        }

        tier_bounds = {
            "Low-Tier":      (ymin, 0.79),
            "Mid-Tier":      (0.8, 1.19),
            "Premium":       (1.2, min(1.79, ymax)),
            "Super Premium": (min(1.8, ymax), ymax),
        }

        has_brands = {}
        for tier, constraint in tier_constraints.items():
            has_brands[tier] = constraint(rel_price.fillna(pd.NA)).fillna(False).any()

        tier_shapes = {}
        for shp in list(shapes):
            if getattr(shp, "has_text_frame", False):
                name = shp.text_frame.text.strip()
        
                if name in tier_constraints:
                    keep = False
                    if has_brands[name]:
                        keep = True
                    else:
                        if name == "Mid-Tier":
                            if has_brands.get("Low-Tier", False) and has_brands.get("Premium", False):
                                keep = True
                        elif name == "Premium":
                            if has_brands.get("Mid-Tier", False) and has_brands.get("Super Premium", False):
                                keep = True
                    if keep:
                        tier_shapes[name] = shp
                    else:
                        shp.element.getparent().remove(shp.element)

        # Plot box for positioning tier rectangles
        plot_box = None
        for shp in slide.shapes:
            if shp.name == "PlotAreaBox":
                plot_box = shp
                break

        plot_top = plot_box.top
        plot_height = plot_box.height

        def val_to_slide_y(val):
            # Convert axis value to slide Y coordinate
            val = max(min(val, ymax), ymin)
            rel_pos = (val - ymin) / (ymax - ymin) if ymax > ymin else 0
            return plot_top + plot_height - (rel_pos * plot_height)

        for name, shp in list(tier_shapes.items()):
            low, high = tier_bounds[name]
            low_clip = max(low, ymin)
            high_clip = min(high, ymax)
            if high_clip <= low_clip:
                shp.element.getparent().remove(shp.element)
                continue
            top_y = val_to_slide_y(high_clip)
            bottom_y = val_to_slide_y(low_clip)
            shp.top = int(top_y)
            shp.height = int(bottom_y - top_y)

# Sector/Segment Leadership Table

In [ ]:
# def leadership_table1(prs,modified_price_positioning_sorted,sharGrowthDf_sec,modified_sector_total_leadership, position=0,Inscope=" ",slideby=" "):
    
#     slide_num = 0
#     for key in sharGrowthDf_sec.keys():
#         slide = prs.slides[slide_num +position]
#         tables, charts = createTableAndChart(slide.shapes)

#         shapes = slide.shapes
#         titleNumber = get_shape_number(shapes, "Share and Growth By Brands | By Category | National | P12M")
#         headerNumber = get_shape_number(shapes, "Sector/Segment Leadership Table (Replace with SO WHAT)")

#         shapes[titleNumber-1].text = data_source # Set data source information
#             # Set title for the slide
#         shapes[titleNumber].text = f'Share and Growth By {slideby} | By {Inscope} | {categories[0]} | {key} | P12M'
#         shapes[titleNumber].text_frame.paragraphs[0].font.size = Pt(12)
#         shapes[titleNumber].text_frame.paragraphs[0].font.name = 'Nexa (Headings)'
#         # shapes[headerNumber].text_frame.paragraphs[0].font.size = Pt(16)
#         # shapes[headerNumber].text_frame.paragraphs[0].font.name = 'Nexa Bold'
#         i =1
#         no_of_tables =len(sharGrowthDf_sec[key])
#         for sec in sharGrowthDf_sec[key]:
#             if i >= len(tables):
#                 continue
#             df = modified_price_positioning_sorted[key + ' | '+sec]
#             df = df[df['Value Share'] >0.01].reset_index(drop=True)
#             table1 = tables[i].table
#             num_rows_to_remove = len(table1.rows) - df.shape[0] - 1
#             table1 = removeRowFromTable(table1, num_rows_to_remove, rowToExclude=1)
#             for row_number, row in enumerate(table1.rows, start=0):
#                 for column_num, cell in enumerate(row.cells):
#                     if row_number ==0 and column_num ==0:
#                         # r=  round(modified_sector_total_leadership[key].loc[modified_sector_total_leadership[key][Inscope] == sec, "WoB %"] * 100)
#                         wob_series = modified_sector_total_leadership[key].loc[
#                             modified_sector_total_leadership[key][Inscope] == sec, "WoB %"
#                         ].fillna(0)

#                         # Ensure there is at least one value before accessing
#                         if not wob_series.empty:
#                             catwob = int(round(wob_series.iloc[0] * 100, 0))
#                         else:
#                             catwob = 0             
#                         cell.text = f"{sec} ({catwob}%)"
#                         set_cell_font(cell, 'Nexa Bold', 8)
#                         cell.text_frame.paragraphs[0].alignment = PP_ALIGN.CENTER
#                         cell.text_frame.paragraphs[0].font.bold = True
#                         cell.text_frame.paragraphs[0].font.color.rgb = RGBColor(87, 85, 85)
#                     if row_number ==0 and column_num ==3:
#                         currencywithoutspace =currency.strip()  # Remove the leading space
#                         cell.text = f"Avg Price /Kg ({currencywithoutspace})"
#                         set_cell_font(cell, 'Nexa Bold', 8)
#                         cell.text_frame.paragraphs[0].alignment = PP_ALIGN.CENTER
#                         cell.text_frame.paragraphs[0].font.bold = True
#                         cell.text_frame.paragraphs[0].font.color.rgb = RGBColor(87, 85, 85)

#                     if row_number >0 and column_num == 0:
#                         cell.text = df[slideby][row_number -1]
#                         set_cell_font(cell, 'Nexa Bold', 8)
#                     elif row_number >0 and column_num == 1:
#                         cell.text = str(format_number(df['Value Share'][row_number -1]*100, use_decimals=True, decimals=1))+'%'
#                         set_cell_font(cell, 'Nexa Book', 7)
#                     elif row_number >0 and column_num == 2:
#                         value = float(format_number(df['Value Share DYA'][row_number -1]*100, use_decimals=True, decimals=1))
#                         if  value > 0.2:
#                              cell.text = str(value) +'%'
#                              cell.text_frame.paragraphs[0].font.color.rgb = RGBColor(0, 160, 151)
#                         elif value < - 0.2:
#                             cell.text = str(value) +'%'
#                             cell.text_frame.paragraphs[0].font.color.rgb = RGBColor(192, 0, 0)
#                         else:
#                             cell.text = str(value) +'%'
#                             cell.text_frame.paragraphs[0].font.color.rgb = RGBColor(14, 40, 65)
                            
#                         set_cell_font(cell, 'Nexa Book', 7)
#                     elif row_number >0 and column_num == 3:
#                         cell.text = str(format_number(df['Av Price/KG'][row_number -1], use_decimals=True, decimals=2))
#                         set_cell_font(cell, 'Nexa Book', 7)
#                     elif row_number >0 and column_num == 4:
#                         price = df['IYA Price/KG'][row_number - 1]
#                         if pd.isna(price):
#                             cell.text = ""
#                         else:
#                             cell.text = str(round(df['IYA Price/KG'][row_number -1]*100))
#                         set_cell_font(cell, 'Nexa Book', 7) 
#             i+=1
#             for table_index in range(len(tables),no_of_tables,-1):
#                 remove_specific_table_from_slide(slide, table_index)

#         slide_num +=1

In [ ]:
def leadership_table1(prs, modified_price_positioning_sorted, sharGrowthDf_sec,
                      modified_sector_total_leadership, position=0, Inscope=" ", slideby=" "):
    
    keys = list(sharGrowthDf_sec.keys())

    slide_num = 0  # Used to shift through slides
    for key in keys:
        grouped_subsegments = sharGrowthDf_sec[key]  # e.g. [['A', 'B'], ['C', 'D']]
        for group in grouped_subsegments:
            # Each group gets its own slide
            slide = prs.slides[slide_num + position]
            tables, charts = createTableAndChart(slide.shapes)
            shapes = slide.shapes

            titleNumber = get_shape_number(shapes, "Share and Growth By Brands | By Category | National | P12M")
            headerNumber = get_shape_number(shapes, "Sector/Segment Leadership Table (Replace with SO WHAT)")

            shapes[titleNumber - 1].text = data_source
            shapes[titleNumber].text = f'Share and Growth By {slideby} | By {Inscope} | {categories[0]} | {key} | P12M'
            shapes[titleNumber].text_frame.paragraphs[0].font.size = Pt(12)
            shapes[titleNumber].text_frame.paragraphs[0].font.name = 'Nexa (Headings)'

            i = 1
            for sec in group:
                if i >= len(tables):
                    continue

                df_key = sec + ' | ' + key
                if df_key not in modified_price_positioning_sorted:
                    continue

                df = modified_price_positioning_sorted[df_key]
                df = df[df['Value Share'] > 0.01].reset_index(drop=True)

                table1 = tables[i].table
                num_rows_to_remove = len(table1.rows) - df.shape[0] - 1
                table1 = removeRowFromTable(table1, num_rows_to_remove, rowToExclude=1)

                for row_number, row in enumerate(table1.rows, start=0):
                    for column_num, cell in enumerate(row.cells):
                        if row_number == 0 and column_num == 0:
                            wob_series = modified_sector_total_leadership[key].loc[
                                modified_sector_total_leadership[key][Inscope] == sec, "WoB %"
                            ].fillna(0)
                            catwob = int(round(wob_series.iloc[0] * 100, 0)) if not wob_series.empty else 0
                            cell.text = f"{sec} ({catwob}%)"
                            set_cell_font(cell, 'Nexa Bold', 8)
                            cell.text_frame.paragraphs[0].alignment = PP_ALIGN.CENTER
                            cell.text_frame.paragraphs[0].font.bold = True
                            cell.text_frame.paragraphs[0].font.color.rgb = RGBColor(87, 85, 85)

                        elif row_number == 0 and column_num == 3:
                            currencywithoutspace = currency.strip()
                            cell.text = f"Avg Price /Vol ({currencywithoutspace})"
                            set_cell_font(cell, 'Nexa Bold', 8)
                            cell.text_frame.paragraphs[0].alignment = PP_ALIGN.CENTER
                            cell.text_frame.paragraphs[0].font.bold = True
                            cell.text_frame.paragraphs[0].font.color.rgb = RGBColor(87, 85, 85)

                        elif row_number == 0 and column_num == 4:
                            currencywithoutspace = currency.strip()
                            cell.text = f"IYA Price /Vol ({currencywithoutspace})"
                            set_cell_font(cell, 'Nexa Bold', 8)
                            cell.text_frame.paragraphs[0].alignment = PP_ALIGN.CENTER
                            cell.text_frame.paragraphs[0].font.bold = True
                            cell.text_frame.paragraphs[0].font.color.rgb = RGBColor(87, 85, 85)

                        elif row_number > 0:
                            if column_num == 0:
                                cell.text = df[slideby][row_number - 1]
                                set_cell_font(cell, 'Nexa Bold', 8)
                            elif column_num == 1:
                                cell.text = str(format_number(df['Value Share'][row_number - 1] * 100, use_decimals=True, decimals=1)) + '%'
                                set_cell_font(cell, 'Nexa Book', 7)
                            elif column_num == 2:
                                value = float(format_number(df['Value Share DYA'][row_number - 1] * 100, use_decimals=True, decimals=1))
                                sign = "+" if value > 0 else ""
                                cell.text = f"{sign}{value}%"
                                if value > 0.2:
                                    cell.text_frame.paragraphs[0].font.color.rgb = RGBColor(0, 160, 151)
                                elif value < -0.2:
                                    cell.text_frame.paragraphs[0].font.color.rgb = RGBColor(192, 0, 0)
                                else:
                                    cell.text_frame.paragraphs[0].font.color.rgb = RGBColor(14, 40, 65)
                                set_cell_font(cell, 'Nexa Book', 7)
                            elif column_num == 3:
                                cell.text = str(format_number(df['Av Price/KG'][row_number - 1], use_decimals=True, decimals=2))
                                set_cell_font(cell, 'Nexa Book', 7)
                            elif column_num == 4:
                                price = df['IYA Price/KG'][row_number - 1]
                                cell.text = "" if pd.isna(price) else str(round(price * 100))
                                set_cell_font(cell, 'Nexa Book', 7)
                i += 1

            # Delete any extra unused tables from the slide
            for table_index in range(len(tables), len(group), -1):
                remove_specific_table_from_slide(slide, table_index)

            slide_num += 1


# Sector Segment Leadership Analysis 2 levels

In [ ]:
def totalSegmentLeadership(prs,numOfDuplicates,totalDf,brandDf,segmentInScope,position=0, parent = 'Sector', child = 'Segment',slideby=""):
    """
    Generate slides for total segment leadership analysis.

    Args:
        prs: PowerPoint presentation object.
        numOfDuplicates: Number of duplicate slides to generate.
        totalDf: DataFrame containing total segment data.
        brandDf: Dictionary containing brand-specific segment data.
        segmentInScope: List of segments to include in the analysis.
        position: Position index to start adding slides (default is 0).
    """
    for slidenum in range(numOfDuplicates):
        # Extract market and total market name
        market=list(brandDf.keys())[slidenum]

        totalMarket=market.split(' | ')[1]
        # Access shapes in the slide
        shapes = prs.slides[slidenum+position].shapes
        titleNumber = get_shape_number(shapes, "Segments Value Sales & Avg Price Per Liter | Category vs. Brand A | National | P12M")
        headerNumber = get_shape_number(shapes, "Segment Leadership Analysis (Replace with So What) ")
        # Update text boxes in the slide
        currencywithoutspace =currency.strip()  # Remove the leading space

        shapes[8].text = f"Value Sales \n (M {currencywithoutspace})"
        shapes[9].text = f"Av. Price/Vol \n ({currencywithoutspace})"

        # Access the text frame and iterate over all paragraphs
        text_frame = shapes[8].text_frame
        text1_frame = shapes[9].text_frame
        # Iterate over all paragraphs in both text frames
        for frame in [text_frame, text1_frame]:
            for paragraph in frame.paragraphs:
                # Set font properties for each paragraph
                paragraph.font.size = Pt(8)
                paragraph.font.bold = True
                paragraph.font.name = 'Nexa Bold'  # Set font to 'Nexa Bold'
                paragraph.font.color.rgb = RGBColor(87, 85, 85)  # Set color to gray

                # Set text alignment based on the text frame
                if frame is text_frame:
                    paragraph.alignment = PP_ALIGN.RIGHT
                else:
                    paragraph.alignment = PP_ALIGN.LEFT
        shapes[titleNumber-1].text = data_source

        shapes[titleNumber].text = f'{child} Value Sales & Avg Price Per Vol | Category vs. '+market.split(' | ')[0]+' | '+totalMarket+' |  P12M' 
    
        # shapes[titleNumber].text_frame.paragraphs[0].font.bold = True
        shapes[titleNumber].text_frame.paragraphs[0].font.size = Pt(12)
        shapes[titleNumber].text_frame.paragraphs[0].font.name = 'Nexa (Headings)'     
        tables,charts=createTableAndChart(shapes)
        
        totalDf[totalMarket]=totalDf[totalMarket][totalDf[totalMarket][child].isin(segmentInScope)]
        
        brandDf[market]=brandDf[market][brandDf[market][child].isin(segmentInScope)]
        #if "National" not in market: print(brandDf[totalMarket])

        rest=totalDf[totalMarket][~totalDf[totalMarket][child].isin(brandDf[market][child])]
        rest[['Value Sales','Av Price/KG','WoB %','Gross Margin %']]=0
        df=pd.concat([rest,brandDf[market]]).sort_values(by=[f'{parent}',f'{child}'])
        df=df[df[child].isin(segmentInScope)]
        df=totalDf[totalMarket].merge(df,on=[f'{parent}',f'{child}'],suffixes=('','_Brand'),how='left').sort_values(by=[f'{parent}','Value Sales',f'{child}'],ascending=[True,False,True]).reset_index(drop=True)
        df = df.replace([np.nan, np.inf, -np.inf], 0)

        dfTotal=df[[f'{parent}',f'{child}', 'Value Sales', 'Av Price/KG', 'WoB %','Gross Margin %']]
        dfBrand=df[[f'{parent}',f'{child}', 'Value Sales_Brand', 'Av Price/KG_Brand','WoB %_Brand', 'Gross Margin %_Brand']]
        dfBrand.columns=dfBrand.columns.str.replace('_Brand','')
        table=tables[0].table
        num_columns_to_remove = (len(table.columns) - dfTotal.shape[0]) - 1  # Specify the number of rows to remove from the end
        table_width = Inches(8.49)  # Specify the desired table height
        

        # table=col_cell_remove(table,num_columns_to_remove,table_width,dfTotal)
        table=col_cell_remove(table,num_columns_to_remove)

        for i, row in enumerate(table.rows):
            for j, cell in enumerate(row.cells):
                if slideby=="Top Companies":
                    if j==0 and i==1:
                        cell.text="Company WoB%"
                        cell.text_frame.paragraphs[0].runs[0].font.color.rgb = RGBColor(255, 255, 255)  # Set text color to white
                        cell.text_frame.paragraphs[0].runs[0].font.size = Pt(7)
                if j!=0:
                    if i==0:
                        cell.text = str((round(dfTotal['WoB %'].iloc[j-1]*100,1)))+'%' if round(dfTotal['WoB %'].iloc[j-1]*100,1)!=0 else ''
                    else:
                        if len(dfBrand['WoB %'])<j:
                            cell.text=''
                        else:
                            cell.text = str((round(dfBrand['WoB %'].iloc[j-1]*100,1)))+'%' if round(dfBrand['WoB %'].iloc[j-1]*100,1)!=0 else ''
                    if cell.text!='':
                        cell.text_frame.paragraphs[0].alignment = PP_ALIGN.CENTER
                        cell.text_frame.paragraphs[0].runs[0].font.size = Pt(7)
                        cell.text_frame.paragraphs[0].runs[0].font.bold = False
                        cell.text_frame.paragraphs[0].font.name = 'Nexa Book'

        table=tables[1].table
        # table=col_cell_remove(table,num_columns_to_remove,table_width,dfTotal)
        table=col_cell_remove(table,num_columns_to_remove)

        for i, row in enumerate(table.rows):
            for j, cell in enumerate(row.cells):
                if slideby==f"{ManufOrTopC}":
                    if j==0 and i==0:
                        cell.text="Company GM%"
                        # cell.text_frame.paragraphs[0].alignment = PP_ALIGN.CENTER
                        cell.text_frame.paragraphs[0].runs[0].font.size = Pt(7)
                        cell.text_frame.paragraphs[0].runs[0].font.color.rgb = RGBColor(255, 255, 255)  # Set text color to white

                if j!=0:
                    if i==0:
                        cell.text = str((round(dfBrand['Gross Margin %'].iloc[j-1]*100,1)))+'%' if round(dfBrand['Gross Margin %'].iloc[j-1]*100,1)!=0 else ''
                    else:
                        if len(dfBrand['Gross Margin %'])<j:
                            cell.text=''
                        else:
                            cell.text = str((round(dfBrand['Gross Margin %'].iloc[j-1]*100,1)))+'%' if round(dfBrand['Gross Margin %'].iloc[j-1]*100,1)!=0 else ''

                    if cell.text!='':

                        cell.text_frame.paragraphs[0].alignment = PP_ALIGN.CENTER
                        cell.text_frame.paragraphs[0].runs[0].font.size = Pt(7)
                        cell.text_frame.paragraphs[0].runs[0].font.bold = False
                        cell.text_frame.paragraphs[0].font.name = 'Nexa Book'
        
        chart=charts[0].chart

        catLevelOne=['Total']+[market.split(' | ')[0]]
        secLevels=dfTotal.groupby([f'{parent}'])[f'{child}'].unique().reset_index()
        secLevels=dict(zip(secLevels[parent],secLevels[child]))
        catLevels=secLevels.copy()

        chart_data = CategoryChartData()
        categories = chart_data.categories
        for level3 in catLevels.keys():
            category_3 = categories.add_category(level3)
            for level2 in catLevels[level3]:
                
                category_2=category_3.add_sub_category(level2)
                for level1 in catLevelOne:
                    
                    category_2.add_sub_category(level1)
        vs=[[df['Value Sales'][i],df['Value Sales_Brand'][i]] for i in range(len(df['Value Sales_Brand']))]
        vs= [round(float(item)/10**6,1) for sublist in vs for item in sublist]

        av=[[df['Av Price/KG'].replace(np.nan,0)[i],df['Av Price/KG_Brand'].replace(np.nan,0)[i]] for i in range(len(df['Av Price/KG_Brand']))]
        av= [float(item) for sublist in av for item in sublist]

        chart_data.add_series('Value Sales',vs) 
        chart_data.add_series('Av Price/KG',av)
        chart.replace_data(chart_data)
        
        for i,series in enumerate(chart.series):
            if series.name=='Av Price/KG':
                for j, point in enumerate(series.points):
                    data_label = point.data_label
                    data_label.position=XL_LABEL_POSITION.ABOVE
                    data_label.has_text_frame = True
                    data_label.text_frame.text =str(round(series.values[j], decimals))

        secondary_value_axis = chart.value_axis
        if decimals == 2:
            secondary_value_axis.tick_labels.number_format = f'#,##0.00'
        else:
            secondary_value_axis.tick_labels.number_format = f'#,##0'
            
        secondary_value_axis.auto_axis = True

        for j, point in enumerate(chart.series[0].points):
            if j>len(df['Value Sales'])-1:
                    break
            point.format.fill.solid()
            if j%2==0:
                #Gray Total
                point.format.fill.fore_color.rgb = RGBColor(174, 171, 171)
                
            else:
                # Brand Color
                point.format.fill.fore_color.rgb = RGBColor(203, 234, 231)
        chart.replace_data(chart_data)


# Sector Segment Leadership Analysis 1 level


In [ ]:
def brandSegmentLeadership(prs,numOfDuplicates,totalDf,brandDf,segmentInScope,position=0, slide_by = 'Sectors', child = 'Segment'):
    """
    Generate slides for brand segment leadership analysis.
    Args:
        prs: PowerPoint presentation object.
        numOfDuplicates: Number of duplicate slides to generate.
        totalDf: DataFrame containing total segment data.
        brandDf: Dictionary containing brand-specific segment data.
        segmentInScope: List of segments to include in the analysis.
        position: Position index to start adding slides (default is 0).
        slide_by: Type of segmentation for the analysis (default is 'Sectors').
    """
    for slidenum in range(numOfDuplicates):
         # Extract market and total market name
        market=list(brandDf.keys())[slidenum]
        totalMarket=market.split(' | ')[1]

        shapes = prs.slides[slidenum+position].shapes
        titleNumber = get_shape_number(shapes, "Sectors Value Sales & Avg Price Per Liter | Category vs. Brand A | National | P12M")
        headerNumber = get_shape_number(shapes, "Sectors Leadership Analysis (Replace with So What)")
        currencywithoutspace =currency.strip()  # Remove the leading space

        shapes[8].text = f"Value Sales \n (M {currencywithoutspace})"
        shapes[9].text = f"Av. Price/Vol \n ({currencywithoutspace})"

        # Access the text frame and iterate over all paragraphs
        text_frame = shapes[8].text_frame
        text1_frame = shapes[9].text_frame


        # Iterate over all paragraphs in both text frames
        for frame in [text_frame, text1_frame]:
            for paragraph in frame.paragraphs:
                # Set font properties for each paragraph
                paragraph.font.size = Pt(8)
                paragraph.font.bold = True
                paragraph.font.name = 'Nexa Bold'  # Set font to 'Nexa Bold'
                paragraph.font.color.rgb = RGBColor(87, 85, 85)  # Set color to gray

                # Set text alignment based on the text frame
                if frame is text_frame:
                    paragraph.alignment = PP_ALIGN.RIGHT
                else:
                    paragraph.alignment = PP_ALIGN.LEFT
        shapes[titleNumber-1].text = data_source
        shapes[titleNumber].text = f'{slide_by} Value Sales & Avg Price Per Vol | Category vs. '+market.split(' | ')[0]+' | '+totalMarket+' |  P12M' 
        shapes[headerNumber].text = f"{slide_by} Leadership Analysis"
        # shapes[titleNumber].text_frame.paragraphs[0].font.bold = True
        shapes[titleNumber].text_frame.paragraphs[0].font.size = Pt(12)
        shapes[titleNumber].text_frame.paragraphs[0].font.name = 'Nexa (Headings)'         # Extract tables and charts from shapes
        tables,charts=createTableAndChart(shapes)
       
        table=tables[0].table
        rest=totalDf[totalMarket][~totalDf[totalMarket][child].isin(brandDf[market][child])]
        rest[['Value Sales','Av Price/KG','WoB %','Gross Margin %']]=0
        df=pd.concat([rest,brandDf[market]]).sort_values(by=[f'{child}'])
        df = df.replace([np.nan, np.inf, -np.inf], 0)
        # Merge total and brand-specific dataframes
        df=totalDf[totalMarket].merge(df,on=[f'{child}'],suffixes=('','_Brand'),how='left').sort_values(by=['Value Sales'],ascending=[False]).reset_index(drop=True)
        dfTotal=df[[f'{child}', 'Value Sales', 'Av Price/KG', 'WoB %','Gross Margin %']]
        
        dfBrand=df[[f'{child}', 'Value Sales_Brand', 'Av Price/KG_Brand','WoB %_Brand', 'Gross Margin %_Brand']]
        
        dfBrand[['Value Sales_Brand', 'Av Price/KG_Brand','WoB %_Brand', 'Gross Margin %_Brand']]=dfBrand[['Value Sales_Brand', 'Av Price/KG_Brand','WoB %_Brand', 'Gross Margin %_Brand']].replace(np.nan,0)
        dfTotal[['Value Sales', 'Av Price/KG','WoB %', 'Gross Margin %']]=dfTotal[['Value Sales', 'Av Price/KG','WoB %', 'Gross Margin %']].replace(np.nan,0)
        dfBrand.columns=dfBrand.columns.str.replace('_Brand','')
        num_columns_to_remove = (len(table.columns) - dfTotal.shape[0]) - 1  # Specify the number of rows to remove from the end
        table_width = Inches(8.51)  # Specify the desired table height


        # table=col_cell_remove(table,num_columns_to_remove,table_width,dfTotal)
        table=col_cell_remove(table,num_columns_to_remove)

        for i, row in enumerate(table.rows):
            for j, cell in enumerate(row.cells):
                if j!=0:
                    if i==0:
                        cell.text = str((round(dfTotal['WoB %'].iloc[j-1]*100,1)))+'%' if round(dfTotal['WoB %'].iloc[j-1]*100,1)!=0 else ''
                    else:
                        if len(dfBrand['WoB %'])<j:
                            cell.text=''
                        else:
                            cell.text = str((round(dfBrand['WoB %'].iloc[j-1]*100,1)))+'%' if round(dfBrand['WoB %'].iloc[j-1]*100,1)!=0 else ''
                    if cell.text!='':
                        cell.text_frame.paragraphs[0].alignment = PP_ALIGN.CENTER
                        cell.text_frame.paragraphs[0].runs[0].font.size = Pt(7)
                        cell.text_frame.paragraphs[0].runs[0].font.bold = False
                        cell.text_frame.paragraphs[0].font.name = 'Nexa Book'

        table=tables[1].table
        num_columns_to_remove = (len(table.columns) - dfTotal.shape[0]) - 1  # Specify the number of rows to remove from the end

        # table=col_cell_remove(table,num_columns_to_remove,table_width,dfTotal)
        table=col_cell_remove(table,num_columns_to_remove)

        for i, row in enumerate(table.rows):
            for j, cell in enumerate(row.cells):
                if j!=0:
                    if i==0:
                        cell.text = str((round(dfBrand['Gross Margin %'].iloc[j-1]*100,1)))+'%' if round(dfBrand['Gross Margin %'].iloc[j-1]*100,1)!=0 else ''
                    else:
                        if len(dfBrand['Gross Margin %'])<j:
                            cell.text=''
                        else:
                            cell.text = str((round(dfBrand['Gross Margin %'].iloc[j-1]*100,1)))+'%' if round(dfBrand['Gross Margin %'].iloc[j-1]*100,1)!=0 else ''

                    if cell.text!='':
                        cell.text_frame.paragraphs[0].alignment = PP_ALIGN.CENTER
                        cell.text_frame.paragraphs[0].runs[0].font.size = Pt(7)
                        cell.text_frame.paragraphs[0].runs[0].font.bold = False
                        cell.text_frame.paragraphs[0].font.name = 'Nexa Book'
        chart=charts[0].chart

        catLevelOne=['Total']+[market.split(' | ')[0]]
        
        
        secLevels=dfTotal[child].unique()
        catLevels=secLevels.copy()
        chart_data = CategoryChartData()
        categories = chart_data.categories
        for level3 in catLevels:
            category_3 = categories.add_category(level3)
            for level2 in catLevelOne:
                category_2=category_3.add_sub_category(level2)
                
        df=df.replace(np.nan,'0')
        vs=[[df['Value Sales'][i],df['Value Sales_Brand'][i]] for i in range(len(df['Value Sales_Brand']))]
        vs= [round(float(item)/10**6,1) for sublist in vs for item in sublist]
        av=[[df['Av Price/KG'][i],df['Av Price/KG_Brand'][i]] for i in range(len(df['Av Price/KG_Brand']))]
        av= [float(item) for sublist in av for item in sublist]
        
        chart_data.add_series('Value Sales',vs) 
        chart_data.add_series('Av Price/KG',av)
        chart.replace_data(chart_data)
        
        for i,series in enumerate(chart.series):
            if series.name=='Av Price/KG':
                for j, point in enumerate(series.points):      
                    data_label = point.data_label
                    data_label.has_text_frame = True
                    data_label.text_frame.text = str(round(series.values[j], decimals))
                        
                    data_label.position=XL_LABEL_POSITION.ABOVE
        chart.secondary_value_axis = chart.value_axis
        
        # Now you can set properties for the secondary axis
        secondary_value_axis = chart.secondary_value_axis
        secondary_value_axis.tick_labels.number_format = f'#,##0.00' if decimals == 2 else f'#,##0'
        secondary_value_axis.auto_axis = True

        for j, point in enumerate(chart.series[0].points):
            if j>len(df['Value Sales'])-1:
                    break
            point.format.fill.solid()
            if j%2==0:
                #Gray Total
                point.format.fill.fore_color.rgb = RGBColor(174, 171, 171)
                
            else:
                # Brand Color
                point.format.fill.fore_color.rgb = RGBColor(203, 234, 231)


        chart.replace_data(chart_data)

# Shelf price/ vol & Avg price/ vol Comparison

In [ ]:
def shelfPrice_AvgPrice(prs,all_brands,numOfDuplicates,salesColumn,sec_seg='Sector',position=0,brandORmanuf=""):
        """
        Add average price/volume charts and tables to PowerPoint slides.

        Args:
        - prs (Presentation): PowerPoint presentation object.
        - all_brands (dict): A dictionary where each key is a string indicating the market or category name, and the value is a DataFrame containing brand information.
        - numOfDuplicates (int): Number of duplicate slides.
        - salesColumn (str): Name of the column containing sales data.
        - sec_seg (str, optional): The name of the column representing the sector or segment. Defaults to 'Sector'.
        - position (int, optional): Position of the slide. Defaults to 0.
        """
        for slidenum in range(numOfDuplicates):
            # Extract market and brand from keys in all_brands dictionary
            market=list(all_brands.keys())[slidenum].split(" | ",1)[1]
            brandInScope = list(all_brands.keys())[slidenum].split(" | ")[0]
             # Access slide shapes

            shapes = prs.slides[slidenum+position].shapes
            footerNumber = get_shape_number(shapes,"Ix = Price Positioning vs leader brand by Sector")
            if brandORmanuf==f'{BrandOrTopB}':
                shapes[footerNumber].text=f"Ix = Price Positioning vs leader brand by {sec_seg}" 
            else:
                shapes[footerNumber].text=f"Ix = Price Positioning vs leader Company by {sec_seg}" 

            titleNumber = get_shape_number(shapes,"Avg Price/Vol ($) | By Sector | Brand vs. Competition | P12M | National")
            currencywithoutspace =currency.strip()  # Remove the leading space
            if titleNumber is None :
                titleNumber = get_shape_number(shapes,"Shelf Price/Vol ($) | By Sector | Brand vs. Competition | P12M | National")
                shapes[titleNumber].text = f"Shelf Price/Vol ({currencywithoutspace}) | By {sec_seg} | {brandInScope} vs. Competition | P12M | {market}"
            else:
                shapes[titleNumber].text = f"Avg Price/Vol ({currencywithoutspace}) | By {sec_seg} | {brandInScope} vs. Competition | P12M | {market}"

            headerNumber = get_shape_number(shapes, "Avg Price/Vol Comparison (Replace with SO WHAT"or"Shelf Price/Vol Comparison (Replace with SO WHAT)")
            shapes[titleNumber-1].text = data_source
            # shapes[titleNumber].text_frame.paragraphs[0].font.bold = True
            shapes[titleNumber].text_frame.paragraphs[0].font.size = Pt(12)
            shapes[titleNumber].text_frame.paragraphs[0].font.name = 'Nexa (Headings)'            # Create table and chart objects
            tables,charts=createTableAndChart(shapes)
            chart=charts[0].chart
            chart_shape = charts[0]
            chart_shape.width = Inches(9.43)  # Set desired width

            chart_data = CategoryChartData()
            df = all_brands[list(all_brands.keys())[slidenum]]

            # Filter the DataFrame to include only rows where 'Top Brands' is in client_brands
            if brandORmanuf==f'{ManufOrTopC}':
                    client_brand_rows = df[df[brandORmanuf].isin(client_manuf)]
            else :
                client_brand_rows = df[df[brandORmanuf].isin(client_brands)]
            # print(client_brand_rows)
            # # Create a list of 'Ix' values where 'Top Brands' matches client brands
            ix_list = client_brand_rows[f'Ix_{salesColumn}'].tolist()
            if  not ix_list :
                ix_list = ['']  # Set ix_list to an empty string

            # # You can now use this list for chart_data.categories or any other purpose
            # chart_data.categories = ix_list
            chart_data.categories =ix_list

            df = all_brands[list(all_brands.keys())[slidenum]]
            if brandORmanuf==f'{BrandOrTopB}':
                chart_data.add_series('All brands',round(all_brands[list(all_brands.keys())[slidenum]][all_brands[list(all_brands.keys())[slidenum]][brandORmanuf]=='Total'][salesColumn].astype(float),decimals).replace({np.nan: None, 0: None}))
            else:
                
                chart_data.add_series('All Companies',round(all_brands[list(all_brands.keys())[slidenum]][all_brands[list(all_brands.keys())[slidenum]][brandORmanuf]=='Total'][salesColumn].astype(float),decimals).replace({np.nan: None, 0: None}))

            # Add client brand first if it exists
            if brandInScope in df[brandORmanuf].unique():
                chart_data.add_series(brandInScope, round(df[df[brandORmanuf] == brandInScope][salesColumn].astype(float), decimals).replace({np.nan: None, 0: None}))

            # Add remaining brands excluding 'Total' and the client brand
            for brand in df[brandORmanuf].unique():
                if brand == 'Total' or brand == brandInScope:
                    continue
                chart_data.add_series(brand, round(df[df[brandORmanuf] == brand][salesColumn].astype(float), decimals).replace({np.nan: None, 0: None}))
            
            value_axis = chart.value_axis
            value_axis.maximum_scale= df[salesColumn].astype(float).max()+0.5
            value_axis.minimum_scale=  df[salesColumn].astype(float).min()- 0.5

            chart.replace_data(chart_data)
            xlsx_file=BytesIO()
            with chart_data._workbook_writer._open_worksheet(xlsx_file) as (workbook, worksheet):
                chart_data._workbook_writer._populate_worksheet(workbook, worksheet)
                worksheet.write(0, 7, sec_seg)
                worksheet.write_column(1, 7, df[sec_seg].unique(), None)

            chart._workbook.update_from_xlsx_blob(xlsx_file.getvalue())

            series1 = chart.series[1]
            # x= client_brands+client_manuf
            for j, point in enumerate(series1.points):      
                data_label = point.data_label
                data_label.position = -4131  # Left position for most chart types

                for brand in df[brandORmanuf].unique():
                        data_label.text_frame.text = f'{str(ix_list[j])}\n{str(series1.values[j])}'
               

            # # Access all series in the chart
            # all_series = chart.series

            # # Create series2 which will exclude series[0]
            # series2 = [series for i, series in enumerate(all_series) if i != 1]  
            # for series in series2:
            #     for j, point in enumerate(series.points):      
            #         data_label = point.data_label
            #         for brand in df[brandORmanuf].unique():
            #             data_label.text_frame.text =f'{str(series.values[j])}'
           


            # Table #
            table=tables[0].table
            # Filter dataframesbin
            dfTotal=all_brands[list(all_brands.keys())[slidenum]][all_brands[list(all_brands.keys())[slidenum]][brandORmanuf]=='Total']
            dfTotal=dfTotal.replace({np.nan: None})
            dfBrandInScope=all_brands[list(all_brands.keys())[slidenum]][all_brands[list(all_brands.keys())[slidenum]][brandORmanuf]==brandInScope]
            dfBrandInScope=dfBrandInScope.replace({np.nan: None})
            if dfBrandInScope.empty:
                columns_to_fill = [sec_seg, 'Sort_Value']  
                for col in columns_to_fill:
                    if col in dfTotal.columns and col in dfBrandInScope.columns:
                        dfBrandInScope[col] = dfTotal[col]
                dfBrandInScope[brandORmanuf] = brandInScope
                        
                dfBrandInScope.fillna(0, inplace=True)
            num_columns_to_remove = (len(table.columns) - dfTotal.shape[0]) - 1  # Specify the number of rows to remove from the end
            table_width = Inches(9.43)  # Specify the desired table height
            table.columns[0].width= Inches(1.1)#524500

            # table=col_cell_remove(table,num_columns_to_remove,table_width,dfTotal)
            table=col_cell_remove(table,num_columns_to_remove)
            total_col_width = table_width - table.columns[0].width
            num_columns = len(table.columns) - 1  # Exclude the first row

            if num_columns > 0:
                cell_width = total_col_width / num_columns
                for col in range(1, table.columns.__len__()):
                    table.columns[col].width = int(cell_width)
                    
            for i, row in enumerate(table.rows):
                for j, cell in enumerate(row.cells):
                    if j==0:
                        if i ==0:
                            cell.text =" "
                        elif(i==1):
                            cell.text=f"{sec_seg} WoB | DYA"
                        elif (i == 2): 
                            if brandORmanuf==f'{ManufOrTopC}':
                                cell.text = f"Company {sec_seg} Share | DYA"
                            else: 
                                cell.text = f"Brand {sec_seg} Share | DYA"
    
                        cell.text_frame.paragraphs[0].font.name = 'Nexa Bold (Headings)'
                        cell.text_frame.paragraphs[0].font.color.rgb = RGBColor(255, 255, 255)
                        cell.text_frame.paragraphs[0].font.size = Pt(7)
                        cell.text_frame.paragraphs[0].alignment = PP_ALIGN.LEFT
                    else:
                        if i==0:
                            unique_values = df[sec_seg].unique()
                            if j - 1 < len(unique_values):  # Check if the index is within bounds
                                cell.text = str(unique_values[j - 1])
                            else:
                                cell.text = ""  # or any default value you want                            # all_brands[list(all_brands.keys())[slidenum]][sec_seg].unique()

                        if i==1:
                            t1="" if dfTotal['Value Share'].iloc[j-1] is None else str((round(dfTotal['Value Share'].iloc[j-1]*100,1)))+'%' if (round(dfTotal['Value Share'].iloc[j-1]*100,1))!=0 else " "
                            value2 = dfTotal['Value Share DYA'].iloc[j-1]
                            if value2 is None:
                                t2 = ""
                            else:
                                value2 = round(float(value2) * 100, 1)
                                t2 = f"+{value2}%" if value2 > 0 else f"{value2}%" if value2 < 0 else "0.0%"
                            # t2="" if dfTotal['Value Share DYA'].iloc[j-1] is None  else str((round(dfTotal['Value Share DYA'].iloc[j-1]*100,1)))+'%' if (round(dfTotal['Value Share DYA'].iloc[j-1]*100,1))!=0 else "0.0%"
                            cell.text = " " if f"{t1} and {t2}" is None else f"{t1} | {t2}"
                        elif (i==2):
                            t3="" if dfBrandInScope['Value Share'].iloc[j-1] is None  else str((round(dfBrandInScope['Value Share'].astype(float).iloc[j-1]*100,1)))+'%' if (round(dfBrandInScope['Value Share'].astype(float).iloc[j-1]*100,1))!=0 else " "
                            value4 = dfBrandInScope['Value Share DYA'].iloc[j-1]
                            if value4 is None:
                                t4 = ""
                            else:
                                value4 = round(float(value4) * 100, 1)
                                t4 = f"+{value4}%" if value4 > 0 else f"{value4}%" if value4 < 0 else "0.0%"
                            # t4="" if dfBrandInScope['Value Share DYA'].iloc[j-1] is None else str((round(dfBrandInScope['Value Share DYA'].astype(float).iloc[j-1]*100,1)))+'%' if (round(dfBrandInScope['Value Share DYA'].astype(float).iloc[j-1]*100,1))!=0 else "0.0%"                           
                            cell.text = " " if t3 == "" and t4 == "" else f"{t3} | {t4}"

                        if cell.text!='':
                            cell.text_frame.paragraphs[0].alignment = PP_ALIGN.CENTER
                            cell.text_frame.paragraphs[0].runs[0].font.size = Pt(7)
                            cell.text_frame.paragraphs[0].runs[0].font.bold = False
                            cell.text_frame.paragraphs[0].font.name = 'Nexa Book'

# Price Point Distribution Analysis by product P3M P12M

In [ ]:
def pricePoint(prs,sectorPD,numOfDuplicates,months='P12M',sec_seg='Sector',position=0):
     """
        Generate slides displaying price point distribution.
 
        Args:
        - prs: PowerPoint presentation object.
        - sectorPD (dict): Dictionary containing DataFrames with price point information for each sector.
        - brandInScope (str): Name of the brand in focus.
        - numOfDuplicates (int): Number of duplicate slides to generate.
        - months (str): Time period for the data.
        - sec_seg (str): The name of the column representing the sector or segment.
        - position (int): Position of the slide in the presentation.
        """
     for slidenum in range(numOfDuplicates):
        dfName=list(sectorPD.keys())[slidenum]
        shapes = prs.slides[slidenum+position].shapes
        titleNumber = get_shape_number(shapes, "Price Point Distribution | Brand | Category | National | P12M")
        headerNumber = get_shape_number(shapes, "Price Point Distribution Analysis by product (Replace with SO WHAT)")
        axistitle=get_shape_number(shapes,"Base Price/Unit ($)")
        currencywithoutspace =currency.strip()  # Remove the leading space
        shapes[axistitle].text=f'Base Price/Unit ({currencywithoutspace})'
        shapes[axistitle].text_frame.paragraphs[0].font.size = Pt(8)
        shapes[axistitle].text_frame.paragraphs[0].font.name = 'Nexa Bold'
        shapes[axistitle].text_frame.paragraphs[0].font.color.rgb = RGBColor(87, 85, 85)  # Set color to black
 
        shapes[titleNumber-1].text = data_source # Set data source information
        # Set title for the slide
        shapes[titleNumber].text = "Price Point Distribution | " + list(sectorPD.keys())[slidenum] + " | " + months
        # shapes[titleNumber].text_frame.paragraphs[0].font.bold = True
        shapes[titleNumber].text_frame.paragraphs[0].font.size = Pt(12)
        shapes[titleNumber].text_frame.paragraphs[0].font.name = 'Nexa (Headings)'    
        tables,charts=createTableAndChart(shapes)
        chart=charts[0].chart
        chart_data = CategoryChartData()
        chart_data.categories = sectorPD[dfName][prodORitem].unique()
        # formatted_prices = sectorPD[dfName]['Base Price/Unit'].apply(lambda x: f"{currency}{round(x, decimals):,.2f}")
        # chart_data.add_series('Base Price/Unit', formatted_prices)
       
        chart_data.add_series('Base Price/Unit', round(sectorPD[dfName]['Base Price/Unit'].astype(float),decimals))
        chart.replace_data(chart_data)
        chart.has_title = False
 
        for i,series in enumerate(chart.series):
                for j, point in enumerate(series.points):      
                    data_label = point.data_label
                    data_label.has_text_frame = True
                    data_label.text_frame.text = str(round(series.values[j], decimals))
                    data_label.position = XL_LABEL_POSITION.ABOVE  # Set the position above the point
 
        value_axis = chart.value_axis
        value_axis.auto_axis = True
        value_axis.minimum_scale = None
        value_axis.major_unit = None
        value_axis.minor_unit = None
        value_axis.axis_title.text_frame.text=''
        value_axis.tick_labels.number_format = "#,##0.00" if decimals ==2 else "#,##0"
 
        # Table #
        table=tables[0].table
 
        num_columns_to_remove = (len(table.columns) - sectorPD[dfName].shape[0]) - 1  # Specify the number of rows to remove from the end
        table_width = Inches(8.89)  # Specify the desired table height
        # table=col_cell_remove(table,num_columns_to_remove,table_width,sectorPD[dfName])
        table=col_cell_remove(table,num_columns_to_remove)
 
       
       
        total_col_width = table_width - table.columns[0].width
        num_columns = len(table.columns) - 1  # Exclude the first row
 
        if num_columns > 0:
            cell_width = total_col_width / num_columns
            for col in range(1, table.columns.__len__()):
                table.columns[col].width = int(cell_width)
 
        mergedCellDf = sectorPD[dfName][sec_seg].value_counts().reset_index()
        mergedCellDf = mergedCellDf.rename(columns = {"index":f'{sec_seg}'})
        mergedCellDf=mergedCellDf.merge(sectorPD[dfName][[sec_seg,'Sort_Value']].drop_duplicates()).sort_values(['Sort_Value'])
 
        mergedCellDf['mergedCell']=mergedCellDf['count'].cumsum()
        count=1
        for mergedCell in mergedCellDf['mergedCell'].unique():
            table.cell(0,count).merge(table.cell(0,mergedCell))
            count=mergedCell+1
 
        table.height = Inches(1.54)
       
        for i, row in enumerate(table.rows):
            for j, cell in enumerate(row.cells):
                if j == 0:
                    if i == 3:
                        cell.text=f"Base Price/Vol ({currencywithoutspace})"
                        cell.text_frame.paragraphs[0].alignment = PP_ALIGN.CENTER
                        cell.text_frame.paragraphs[0].runs[0].font.size = Pt(6)
                        cell.text_frame.paragraphs[0].runs[0].font.bold = False
                        cell.text_frame.paragraphs[0].font.name = 'Nexa (Headings)'
                    continue
               
                if i==0:
                    cell.text=sectorPD[dfName][sec_seg].iloc[j-1]
                    cell.text_frame.paragraphs[0].alignment = PP_ALIGN.CENTER
                    cell.text_frame.paragraphs[0].runs[0].font.size = Pt(8)
                    cell.text_frame.paragraphs[0].runs[0].font.bold = True
                    cell.text_frame.paragraphs[0].font.name = 'Nexa Bold'
                elif (i==1):
                    cell.text=sectorPD[dfName][f"{prodORitem}"].iloc[j-1]
                    cell.text_frame.paragraphs[0].alignment = PP_ALIGN.CENTER
                    cell.text_frame.paragraphs[0].runs[0].font.size = Pt(6)
                    cell.text_frame.paragraphs[0].runs[0].font.bold = True
                    cell.text_frame.paragraphs[0].font.name = 'Nexa Bold'
                elif (i==2):
                    cell.text=sectorPD[dfName]['Total Size'].iloc[j-1]
                elif (i==3):
                    cell.text=str(round(sectorPD[dfName]['Base Price/KG'].astype(float).iloc[j-1],decimals))
                       
                elif (i==4):
                    cell.text=str(int(round(sectorPD[dfName]['Gross Margin %'].replace(np.nan,'0').astype(float).iloc[j-1]*100,0)))+'%' if int(round(sectorPD[dfName]['Gross Margin %'].replace(np.nan,'0').astype(float).iloc[j-1]*100,0))!=0 else ''
                if cell.text!='' and i!=0 and i!=1:
                    cell.text_frame.paragraphs[0].alignment = PP_ALIGN.CENTER
                    cell.text_frame.paragraphs[0].runs[0].font.size = Pt(7)
                    cell.text_frame.paragraphs[0].runs[0].font.bold = False
                    cell.text_frame.paragraphs[0].font.name = 'Nexa Book'

# Price Point Comparison Analysis by Product

In [ ]:
def pricepoint_comparison(prs, merged_dict, position,slideby="Top Brand"):
    for slidenum in range(len(merged_dict)):
            dfName=list(merged_dict.keys())[slidenum]
            market=list(merged_dict.keys())[slidenum].split(' | ')[2]
            category=list(merged_dict.keys())[slidenum].split(' | ')[0]
            # Access shapes in the slide
            shapes = prs.slides[slidenum+position].shapes
            titleNumber = get_shape_number(shapes, "Price Point Distribution | Brand Vs. Competition | Category | National | P3M")
            headerNumber = get_shape_number(shapes, "Price Point Comparison Analysis by Product")
            axistitle=get_shape_number(shapes,"Base Price/Unit ($)")
            currencywithoutspace =currency.strip()  # Remove the leading space
            shapes[axistitle].text=f'Base Price/Unit ({currencywithoutspace})'
            shapes[axistitle].text_frame.paragraphs[0].font.size = Pt(8)
            shapes[axistitle].text_frame.paragraphs[0].font.name = 'Nexa Bold' 
            shapes[axistitle].text_frame.paragraphs[0].font.color.rgb = RGBColor(87, 85, 85)  # Set color to black

            shapes[5].text = data_source # Set data source information
            # Set title for the slide
            shapes[6].text = "Price Point Distribution | " + list(merged_dict.keys())[slidenum].split(' | ')[2] + " Vs. Competition | "+ list(merged_dict.keys())[slidenum].split(' | ')[0]+ ' | ' + list(merged_dict.keys())[slidenum].split(' | ')[1]+ ' | P3M'
            # shapes[titleNumber].text_frame.paragraphs[0].font.bold = True
            shapes[6].text_frame.paragraphs[0].font.size = Pt(12)
            shapes[6].text_frame.paragraphs[0].font.name = 'Nexa (Headings)'

            tables,charts=createTableAndChart(shapes)
            chart=charts[0].chart
            chart_data = CategoryChartData()
            chart_data.categories = merged_dict[dfName][f'{prodORitem}'].unique()        
            chart_data.add_series('Base Price/Unit', round(merged_dict[dfName]['Base Price/Unit'].astype(float),decimals))
            chart.replace_data(chart_data) 
            chart.has_title = False
                
            for i,series in enumerate(chart.series):
                    for j, point in enumerate(series.points):      
                        data_label = point.data_label
                        data_label.has_text_frame = True
                        data_label.text_frame.text = str(round(series.values[j], decimals)) 
                            
            value_axis = chart.value_axis
            value_axis.auto_axis = True
            value_axis.minimum_scale = None
            value_axis.major_unit = None
            value_axis.minor_unit = None
            value_axis.axis_title.text_frame.text=''
            value_axis.tick_labels.number_format = "#,##0.00" if decimals ==2 else "#,##0"

            # Table #
            table=tables[0].table
            num_columns_to_remove = (len(table.columns) - merged_dict[dfName].shape[0]) - 1  # Specify the number of rows to remove from the end
            table_width = Inches(8.89)  # Specify the desired table height
            # table=col_cell_remove(table,num_columns_to_remove,table_width,sectorPD[dfName])
            table=col_cell_remove(table,num_columns_to_remove)
            
            
            total_col_width = table_width - table.columns[0].width
            num_columns = len(table.columns) - 1  # Exclude the first row
            if num_columns > 0:
                cell_width = total_col_width / num_columns
                for col in range(1, table.columns.__len__()):
                    table.columns[col].width = int(cell_width)
            if slideby==f'{BrandOrTopB}' :   
                mergedCellDf = merged_dict[dfName][f'{BrandOrTopB}'].value_counts().reset_index()
                #mergedCellDf = mergedCellDf.rename(columns = {"index":f'{sec_seg}', f'{sec_seg}':'count'})
                #mergedCellDf=mergedCellDf.merge(merged_dict[dfName][['Top Brands','Value Share']].drop_duplicates()).sort_values(['Value Share'], ascending= False)

                client_row = mergedCellDf[mergedCellDf[f'{BrandOrTopB}'].isin(client_brands)]
                if not client_row.empty:
                    client_row = client_row.iloc[0]
                    mergedCellDf = mergedCellDf[~mergedCellDf[f'{BrandOrTopB}'].isin(client_brands)]
                    mergedCellDf = pd.concat([pd.DataFrame([client_row]), mergedCellDf], ignore_index=True)
            
            else:        
                mergedCellDf = merged_dict[dfName][slideby].value_counts().reset_index()

                client_row = mergedCellDf[mergedCellDf[slideby].isin(client_manuf)]
                if not client_row.empty:
                    client_row = client_row.iloc[0]
                    mergedCellDf = mergedCellDf[~mergedCellDf[slideby].isin(client_manuf)]
                    mergedCellDf = pd.concat([pd.DataFrame([client_row]), mergedCellDf], ignore_index=True)

            mergedCellDf['mergedCell']=mergedCellDf['count'].cumsum()
            count=1
            for mergedCell in mergedCellDf['mergedCell'].unique():
                
                table.cell(0,count).merge(table.cell(0,mergedCell))
                count=mergedCell+1

            table.height = Inches(1.54)
            
            for i, row in enumerate(table.rows):
                for j, cell in enumerate(row.cells):
                    if j == 0:
                        if i == 3:    
                            cell.text=f"Base Price/Vol ({currencywithoutspace})"
                            cell.text_frame.paragraphs[0].runs[0].font.size = Pt(6)
                            cell.text_frame.paragraphs[0].runs[0].font.bold = True
                            cell.text_frame.paragraphs[0].font.name = 'Nexa (Headings)'
                        continue
                    if i==0:
                        cell.text=merged_dict[dfName][slideby].iloc[j-1]
                        cell.text_frame.paragraphs[0].alignment = PP_ALIGN.CENTER
                        cell.text_frame.paragraphs[0].runs[0].font.size = Pt(8)
                        cell.text_frame.paragraphs[0].runs[0].font.bold = True
                        cell.text_frame.paragraphs[0].font.name = 'Nexa Bold'
                    elif (i==1):
                        product =merged_dict[dfName][f"{prodORitem}"].iloc[j-1]
                        cell.text=product
                        cell.text_frame.paragraphs[0].alignment = PP_ALIGN.CENTER
                        cell.text_frame.paragraphs[0].runs[0].font.size = Pt(6)
                        cell.text_frame.paragraphs[0].runs[0].font.bold = True
                        cell.text_frame.paragraphs[0].font.name = 'Nexa Bold'
                    elif (i==2):
                        cell.text=merged_dict[dfName]['Total Size'].iloc[j-1]
                    elif (i==3):
                        cell.text=str(round(merged_dict[dfName]['Base Price/KG'].astype(float).iloc[j-1],decimals))
                            
                    elif (i==4):
                        if int(round(merged_dict[dfName]['Gross Margin %'].replace(np.nan,'0').astype(float).iloc[j-1]*100,0))!=0:
                             cell.text = str(int(round(merged_dict[dfName]['Gross Margin %'].replace(np.nan,'0').astype(float).iloc[j-1]*100,0)))+'%' 
                        else:
                            cell.text = ''
                    if cell.text!='' and i!=0 and i!=1:
                        cell.text_frame.paragraphs[0].alignment = PP_ALIGN.CENTER
                        cell.text_frame.paragraphs[0].runs[0].font.size = Pt(7)
                        cell.text_frame.paragraphs[0].runs[0].font.bold = False
                        cell.text_frame.paragraphs[0].font.name = 'Nexa Book'

In [ ]:
def pricepoint_comparison_scraped(prs, merged_dict, position,slideby="Top Brand"):
    for slidenum in range(len(merged_dict)):
            dfName=list(merged_dict.keys())[slidenum]
            market=list(merged_dict.keys())[slidenum].split(' | ')[2]
            category=list(merged_dict.keys())[slidenum].split(' | ')[0]
            # Access shapes in the slide
            if (slidenum + position) >= len(prs.slides):
                print(f"Slide index {slidenum + position} out of range. Skipping.")
                continue
            shapes = prs.slides[slidenum+position].shapes
            titleNumber = get_shape_number(shapes, "Price Point Distribution | Brand Vs. Competition | Category | National | P3M")
            headerNumber = get_shape_number(shapes, "Price Point Comparison Analysis by Product")
            axistitle=get_shape_number(shapes,"Base Price/Unit ($)")
            currencywithoutspace =currency.strip()  # Remove the leading space
            shapes[axistitle].text=f'Scraped Av. Price/Unit ({currencywithoutspace})'
            shapes[axistitle].text_frame.paragraphs[0].font.size = Pt(8)
            shapes[axistitle].text_frame.paragraphs[0].font.name = 'Nexa Bold' 
            shapes[axistitle].text_frame.paragraphs[0].font.color.rgb = RGBColor(87, 85, 85)  # Set color to black

            shapes[5].text = data_source # Set data source information
            # Set title for the slide
            shapes[6].text = "Price Point Distribution | " + list(merged_dict.keys())[slidenum].split(' | ')[2] + " Vs. Competition | "+ list(merged_dict.keys())[slidenum].split(' | ')[0]+ ' | ' + list(merged_dict.keys())[slidenum].split(' | ')[1]+ ' | P3M'
            # shapes[titleNumber].text_frame.paragraphs[0].font.bold = True
            shapes[6].text_frame.paragraphs[0].font.size = Pt(12)
            shapes[6].text_frame.paragraphs[0].font.name = 'Nexa (Headings)'

            tables,charts=createTableAndChart(shapes)
            chart=charts[0].chart
            chart_data = CategoryChartData()
            chart_data.categories = merged_dict[dfName][f'{prodORitem}'].unique()        
            chart_data.add_series('Scraped Av. Price/Unit', round(merged_dict[dfName]['Scraped Av. Price/Unit'].astype(float),decimals))
            chart.replace_data(chart_data) 
            chart.has_title = False
                
            for i,series in enumerate(chart.series):
                    for j, point in enumerate(series.points):      
                        data_label = point.data_label
                        data_label.has_text_frame = True
                        data_label.text_frame.text = str(round(series.values[j], decimals)) 
                            
            value_axis = chart.value_axis
            value_axis.auto_axis = True
            value_axis.minimum_scale = None
            value_axis.major_unit = None
            value_axis.minor_unit = None
            value_axis.axis_title.text_frame.text=''
            value_axis.tick_labels.number_format = "#,##0.00" if decimals ==2 else "#,##0"

            # Table #
            table=tables[0].table
            num_columns_to_remove = (len(table.columns) - merged_dict[dfName].shape[0]) - 1  # Specify the number of rows to remove from the end
            table_width = Inches(8.89)  # Specify the desired table height
            # table=col_cell_remove(table,num_columns_to_remove,table_width,sectorPD[dfName])
            table=col_cell_remove(table,num_columns_to_remove)
            
            
            total_col_width = table_width - table.columns[0].width
            num_columns = len(table.columns) - 1  # Exclude the first row
            if num_columns > 0:
                cell_width = total_col_width / num_columns
                for col in range(1, table.columns.__len__()):
                    table.columns[col].width = int(cell_width)
            if slideby==f'{BrandOrTopB}' :   
                mergedCellDf = merged_dict[dfName][f'{BrandOrTopB}'].value_counts().reset_index()
                # print(dfName, mergedCellDf)
                #mergedCellDf = mergedCellDf.rename(columns = {"index":f'{sec_seg}', f'{sec_seg}':'count'})
                #mergedCellDf=mergedCellDf.merge(merged_dict[dfName][['Top Brands','Value Share']].drop_duplicates()).sort_values(['Value Share'], ascending= False)

                client_row = mergedCellDf[mergedCellDf[f'{BrandOrTopB}'].isin(client_brands)]
                if not client_row.empty:
                    client_row = client_row.iloc[0]
                    mergedCellDf = mergedCellDf[~mergedCellDf[f'{BrandOrTopB}'].isin(client_brands)]
                    mergedCellDf = pd.concat([pd.DataFrame([client_row]), mergedCellDf], ignore_index=True)
            
            else:        
                mergedCellDf = merged_dict[dfName][slideby].value_counts().reset_index()

                client_row = mergedCellDf[mergedCellDf[slideby].isin(client_manuf)]
                if not client_row.empty:
                    client_row = client_row.iloc[0]
                    mergedCellDf = mergedCellDf[~mergedCellDf[slideby].isin(client_manuf)]
                    mergedCellDf = pd.concat([pd.DataFrame([client_row]), mergedCellDf], ignore_index=True)

            mergedCellDf['mergedCell']=mergedCellDf['count'].cumsum()
            count=1
            # print(dfName,mergedCellDf['mergedCell'].unique())
            num_cols = len(table.columns)

            for mergedCell in mergedCellDf['mergedCell'].unique():
                if count < num_cols and mergedCell < num_cols:
                    table.cell(0, count).merge(table.cell(0, mergedCell))
                count = mergedCell + 1

            table.height = Inches(1.54)
            
            for i, row in enumerate(table.rows):
                for j, cell in enumerate(row.cells):
                    if j == 0:
                        if i == 3:    
                            cell.text=f"Scraped Av. Price/Vol ({currencywithoutspace})"
                            cell.text_frame.paragraphs[0].runs[0].font.size = Pt(6)
                            cell.text_frame.paragraphs[0].runs[0].font.bold = True
                            cell.text_frame.paragraphs[0].font.name = 'Nexa (Headings)'
                        continue
                    if i==0:
                        cell.text=merged_dict[dfName][slideby].iloc[j-1]
                        cell.text_frame.paragraphs[0].alignment = PP_ALIGN.CENTER
                        cell.text_frame.paragraphs[0].runs[0].font.size = Pt(8)
                        cell.text_frame.paragraphs[0].runs[0].font.bold = True
                        cell.text_frame.paragraphs[0].font.name = 'Nexa Bold'
                    elif (i==1):
                        product =merged_dict[dfName][f"{prodORitem}"].iloc[j-1]
                        cell.text=product
                        cell.text_frame.paragraphs[0].alignment = PP_ALIGN.CENTER
                        cell.text_frame.paragraphs[0].runs[0].font.size = Pt(6)
                        cell.text_frame.paragraphs[0].runs[0].font.bold = True
                        cell.text_frame.paragraphs[0].font.name = 'Nexa Bold'
                    elif (i==2):
                        value = merged_dict[dfName]['Total Size'].iloc[j-1]
                        cell.text = "" if pd.isna(value) else str(value)
                    elif (i==3):
                        val = merged_dict[dfName]['Scraped Av. Price/KG'].iloc[j-1]
                        if pd.isna(val) or float(val) == 0.0:
                            cell.text = ''
                        else:
                            cell.text = str(round(float(val), decimals))
                            
                    elif (i==4):
                        if int(round(merged_dict[dfName]['Gross Margin %'].replace(np.nan,'0').astype(float).iloc[j-1]*100,0))!=0:
                             cell.text = str(int(round(merged_dict[dfName]['Gross Margin %'].replace(np.nan,'0').astype(float).iloc[j-1]*100,0)))+'%' 
                        else:
                            cell.text = ''
                    if cell.text!='' and i!=0 and i!=1:
                        cell.text_frame.paragraphs[0].alignment = PP_ALIGN.CENTER
                        cell.text_frame.paragraphs[0].runs[0].font.size = Pt(7)
                        cell.text_frame.paragraphs[0].runs[0].font.bold = False
                        cell.text_frame.paragraphs[0].font.name = 'Nexa Book'

# Price Distribution By Brands

In [ ]:
def brandPriceDistributionPPTX(prs,brandPriceDistribution,numOfDuplicates,currency='€',position=0):
        """
        Generate PowerPoint slides with brand price distribution data.

        Args:
        - prs: PowerPoint presentation object.
        - brandPriceDistribution (dict): Dictionary containing DataFrames with brand price distribution information.
        - numOfDuplicates (int): Number of slides to generate.
        - currency (str): Currency symbol.
        - position (int): Position index for inserting slides.

        Returns:
        - None
        """        
        for slidenum in range(numOfDuplicates):
            dfName=list(brandPriceDistribution.keys())[slidenum]
            market=dfName.split(' | ')[1]
            cat=dfName.split(' | ')[0]
            
            shapes = prs.slides[slidenum+position].shapes
            titleNumber = get_shape_number(shapes, "Price Distribution By Brand | Total | National | P12M")
            headerNumber = get_shape_number(shapes, "Price Point Distribution Analysis by brand (Replace with SO WHAT)")
            
            currencywithoutspace =currency.strip()  # Remove the leading space
            shapes[7].text=f'Avg Price\nPer Unit ({currencywithoutspace})'
            for paragraph in shapes[7].text_frame.paragraphs:
                paragraph.font.size = Pt(8)
                paragraph.font.name = 'Nexa Bold'
                paragraph.font.color.rgb = RGBColor(87, 85, 85)
            shapes[titleNumber-1].text = data_source
            shapes[titleNumber].text=shapes[4].text.replace('National',market)
            shapes[titleNumber].text=shapes[4].text.replace('Total',cat)
            

            # shapes[titleNumber].text_frame.paragraphs[0].font.bold = True
            shapes[titleNumber].text_frame.paragraphs[0].font.size = Pt(12)
            shapes[titleNumber].text_frame.paragraphs[0].font.name = 'Nexa (Headings)'    
            tables,charts=createTableAndChart(shapes)

            chart=charts[0].chart

            chart_data = CategoryChartData()
            chart_data.categories =brandPriceDistribution[dfName][f'{BrandOrTopB}'].unique()

            for size in brandPriceDistribution[dfName]['Pack Size'].unique():
                chart_data.add_series(size,brandPriceDistribution[dfName][brandPriceDistribution[dfName]['Pack Size']==size]['Av Price/Unit'].replace({np.nan: None, 0: None}))
            
            chart.replace_data(chart_data)
        
            for series in chart.series:
                series.data_labels.show_series_name = True  # Show data labels
                series.data_labels.font.size=Pt(8)
                series.data_labels.font.name='Nexa Book'
            chart.has_title=False    
            value_axis = chart.value_axis
            value_axis.tick_labels.number_format =  f'#,##0.00'  if decimals == 2 else f'#,##0'
                
            value_axis.has_title=False

            chart.replace_data(chart_data)        

# Price Distribution By Brands by sec/seg/subseg/subcat

In [ ]:
def brandSectorPriceDistributionPPTX(prs,brandPriceDistribution,numOfDuplicates,sec_seg='Sector',currency='€',position=0):
    """
    Generate PowerPoint slides for brand price distribution by sector.

    Args:
    - prs: PowerPoint presentation object
    - brandPriceDistribution (dict): Dictionary containing DataFrames with brand price distribution information.
    - numOfDuplicates (int): Number of duplicate slides to generate.
    - sec_seg (str): Name of the segmentation column.
    - currency (str): Currency symbol.
    - position (int): Position index.

    Returns:
    - None
    """
    
    for slidenum in range(numOfDuplicates):
        dfName=list(brandPriceDistribution.keys())[slidenum]
        
        shapes = prs.slides[slidenum+position].shapes
        titleNumber = get_shape_number(shapes, "Price Distribution By Brand | Sector | National | P12M")
        headerNumber = get_shape_number(shapes, "Price Point Distribution by brand by Sector (Replace with SO WHAT)")
        shapes[headerNumber].text = f"Price Point Distribution by brand by {sec_seg}"
        shapes[titleNumber-1].text = data_source
        currencywithoutspace =currency.strip()  # Remove the leading space
        shapes[7].text=f'Avg Price\nPer Unit ({currencywithoutspace})'
        for paragraph in shapes[7].text_frame.paragraphs:
            paragraph.font.size = Pt(8)
            paragraph.font.name = 'Nexa Bold'
            paragraph.font.color.rgb = RGBColor(87, 85, 85)

        shapes[titleNumber].text=shapes[4].text.replace('National',dfName).replace('Sector',sec_seg)
        # shapes[titleNumber].text_frame.paragraphs[0].font.bold = True
        shapes[titleNumber].text_frame.paragraphs[0].font.size = Pt(12)
        shapes[titleNumber].text_frame.paragraphs[0].font.name = 'Nexa (Headings)'    
        tables,charts=createTableAndChart(shapes)
        chart=charts[0].chart

        chart_data = CategoryChartData()

        categories = chart_data.categories
        df=brandPriceDistribution[dfName]
        for brand in df[f'{BrandOrTopB}'].unique():
            category_1 = categories.add_category(brand)
            for segment in df[df[f'{BrandOrTopB}']==brand][sec_seg].unique():
                category_1.add_sub_category(segment)
        uniqueSizes=df['Pack Size'].unique()
        for packSize in uniqueSizes:
            chart_data.add_series(packSize,df[df['Pack Size']==packSize]['Av Price/Unit'].replace({np.nan: None, 0: None}) )
        for series in chart.series:
            series.data_labels.show_series_name = True  # Show data labels
            series.data_labels.font.size=Pt(8)
            series.data_labels.font.name='Nexa Book'
        chart.has_title=False    
        value_axis = chart.value_axis
        
        value_axis.tick_labels.number_format =  f'#,##0.00'  if decimals == 2 else f'#,##0'
            
        value_axis.has_title=False
        chart.replace_data(chart_data)

# Price Correlation Analysis P3Y

In [ ]:
def price_correlation_p3y(prs, modified_price, modified_price_p12m, categories_dict_P3M, position=0):
    slide_num = 0
    for key, df in modified_price.items():
        df.fillna(0, inplace=True)
        slide = prs.slides[slide_num + position]
        # print(key)
        shapes = slide.shapes
        # Update title shape
        # title_text = f"Price Correlation | {key.split(' | ')[2]} vs {key.split(' | ')[3]} | {key.split(' | ')[0]} | {key.split(' | ')[1]} | Weekly Price vs. Share analysis | P3Y"
        shapes[4].text = data_source  # Make sure the index is correct (5 - 1 = 4)
        shapes[5].text = ("Price Correlation | " + key.split(' | ')[2] + " vs " + key.split(' | ')[3] +
                          " | " + key.split(' | ')[0] + " | " + key.split(' | ')[1] +
                          " | Weekly Price vs. Share analysis\n" "   P3Y        P12M")
        shapes[5].text_frame.paragraphs[0].font.size = Pt(12)
        shapes[5].text_frame.paragraphs[0].font.name = 'Nexa (Headings)'
        shapes[6].text_frame.paragraphs[0].font.size = Pt(16)
        shapes[6].text_frame.paragraphs[0].font.name = 'Nexa Bold'
       
        # Create table and chart objects
        tables, charts = createTableAndChart(slide.shapes)
        chart1 = charts[0].chart  # First chart
        chart2 = charts[1].chart  # Second chart
       
        # Extract data for charts
        category = df[f'{BrandOrTopB}_client'].tolist()
        x_values_price = df['Price lx vs PL'].tolist()
        x_values_av = df['Av Price/KG_client'].tolist()
        y_values = df['Volume Share_client'].tolist()
       
        ###################Chart1#################
        chart_data1 = XyChartData()
        series_P3Y = chart_data1.add_series('P3Y Points')
        series_P12M = chart_data1.add_series('P12M Points')
 
        for i in range(len(category)):
            point = (x_values_price[i], y_values[i])
            is_matched = False
            if key in modified_price_p12m:
                if point in zip(
                        modified_price_p12m[key]['Price lx vs PL'],
                        modified_price_p12m[key]['Volume Share_client']):
                    is_matched = True
 
            if is_matched:
                series_P12M.add_data_point(x_values_price[i], y_values[i])
            else:
                series_P3Y.add_data_point(x_values_price[i], y_values[i])
   # Calculate slope and intercept for series_P12M
        x_values_P12M = []
        y_values_P12M= []
        for point in series_P12M:  
            x_values_P12M.append(point.x)
            y_values_P12M.append(point.y)
        intercept = 0  # Default value
        slope = 1  # Default slope to avoid division by zero

        if len(x_values_P12M) > 1 and len(y_values_P12M) > 1 and len(set(x_values_P12M)) > 1:
            slope, intercept = np.polyfit(x_values_P12M, y_values_P12M, 1)  # Linear fit (degree 1)
            # print(intercept,slope)
        category_axis = chart1.category_axis
        category_axis.auto_axis = True
        min_xvalue1 =min(x_values_price)
        max_xvalue1= max(x_values_price)
 
        ###############P3yQ Line#################
        df['End of Week'] = pd.to_datetime(df['End of Week'])
        # Create columns for Year and Quarter
        df['Year'] = df['End of Week'].dt.year
        df['Quarter'] = df['End of Week'].dt.to_period('Q')
        # Group by Year and Quarter, then calculate averages for numerical columns
        avg_df = df.groupby(['Year', 'Quarter']).agg({
            'Volume Share_client': 'mean'}).reset_index()
        max_volume_share_row = avg_df.loc[avg_df['Volume Share_client'].idxmax()]
        z=(max_volume_share_row['Volume Share_client']-intercept)/slope
        line_series = chart_data1.add_series('Custom Linep3yQ')
        line_series.add_data_point(0, max_volume_share_row['Volume Share_client'])
        line_series.add_data_point(z,  max_volume_share_row['Volume Share_client'])
        line2_series = chart_data1.add_series("Custom Line2p3yQ")
        line2_series.add_data_point(z,  max_volume_share_row['Volume Share_client'])
        line2_series.add_data_point(z, 0)
        ###############P12MQ Line#################
        if key in modified_price_p12m :
            df12m=modified_price_p12m[key]
            df12m['End of Week'] = pd.to_datetime(df12m['End of Week'])
            # Create columns for Year and Quarter
            df12m['Year'] = df12m['End of Week'].dt.year
            df12m['Quarter'] = df12m['End of Week'].dt.to_period('Q')
            # Group by Year and Quarter, then calculate averages for numerical columns
            avg_df12m = df12m.groupby(['Year', 'Quarter']).agg({
                'Volume Share_client': 'mean'}).reset_index()
            max_volume_share_row12m = avg_df12m.loc[avg_df12m['Volume Share_client'].idxmax()]
            p12m=(max_volume_share_row12m['Volume Share_client']-intercept)/slope
            line_series = chart_data1.add_series('Custom Linep12mQ')
            line_series.add_data_point(0, max_volume_share_row12m['Volume Share_client'])
            line_series.add_data_point(p12m,  max_volume_share_row12m['Volume Share_client'])
            line2_series = chart_data1.add_series("Custom Line2p12mQ")
            line2_series.add_data_point(p12m,  max_volume_share_row12m['Volume Share_client'])
            line2_series.add_data_point(p12m, 0)      
        else:
            p12m=0
            max_volume_share_row12m = {'Volume Share_client': None}  # Handle missing data gracefully
        ###############P3M Line#################
        if key in categories_dict_P3M and not categories_dict_P3M[key].empty:
            # Calculate the average volume share
            avg_volume_share = categories_dict_P3M[key]["Volume Share_client"].mean()
            x=(avg_volume_share-intercept)/slope
            line_series = chart_data1.add_series('Custom Linep3m')
            line_series.add_data_point(0, avg_volume_share)
            line_series.add_data_point(x, avg_volume_share)
 
            line2_series = chart_data1.add_series('Custom Line2p3m')
            line2_series.add_data_point(x, avg_volume_share)
            line2_series.add_data_point(x, 0)
        else:
            x=0
            avg_volume_share=0
        x_values_P12M.append(p12m)
        x_values_P12M.append(z)
        x_values_P12M.append(x)    
        ###############Regression line####################
        smallest = min(min_xvalue1, x, p12m, z)
        largest = max(max_xvalue1, x, p12m, z)
        # print("min_xvalue1:",min_xvalue1,"x:",x,"p12m:",p12m,"z:",z,"smallest",smallest)
        if smallest <0:
            smallest =0
        category_axis.minimum_scale=round(smallest*0.8)
        category_axis.maximum_scale=round(largest*1.2)
   # Initialize series_Y_hat
        series_Y_Hat = chart_data1.add_series("Y_Hat")
        for x in x_values_P12M:
            y_hat = slope * x + intercept
            series_Y_Hat.add_data_point(x, y_hat)
 
        if key in modified_price_p12m:
            unique_in_df1 = df[~df['End of Week'].isin(modified_price_p12m[key]['End of Week'])]
        else:
            unique_in_df1 = pd.DataFrame(columns=df.columns)  # Initialize as an empty DataFrame with the same columns
        chart1.replace_data(chart_data1)

        unique_in_df1['End of Week'] = pd.to_datetime(unique_in_df1['End of Week'], errors='coerce').dt.strftime('%Y/%m/%d')
 
        min_yvalue =min(y_values)
        max_yvalue =max(y_values)
 
        value_axis = chart1.value_axis
        value_axis.minimum_scale = (min_yvalue * 0.8)
        value_axis.maximum_scale = (max_yvalue * 1.2)
       
        chart1.category_axis.axis_title.text_frame.text=  'Price Ix vs. '+ key.split(' | ')[3]
        chart1.category_axis.axis_title.text_frame.paragraphs[0].font.size = Pt(8)
        chart1.category_axis.axis_title.text_frame.paragraphs[0].font.name = 'Nexa Bold'
        chart1.category_axis.axis_title.text_frame.paragraphs[0].font.bold = False
        chart1.category_axis.axis_title.text_frame.paragraphs[0].font.color.rgb = RGBColor(87, 85, 85)
        chart1.category_axis.tick_labels.number_format = '0.0%'
 
        xlsx_file = BytesIO()
        # Ensure x is iterable
        with chart_data1._workbook_writer._open_worksheet(xlsx_file) as (workbook, worksheet):
            chart_data1._workbook_writer._populate_worksheet(workbook, worksheet)
            worksheet.write(0, 4, "End of Week")
            worksheet.write_column(1, 4, unique_in_df1['End of Week'], None)
            worksheet.write(0, 7, "Highest Value")
            worksheet.write_column(2, 7, ["P3M Avg"], None)
            worksheet.write_column(3, 7, ["P12M HighestQ"], None)
            worksheet.write_column(4, 7, ["P3Y HighestQ"], None)
            # Write to worksheet
            worksheet.write_column(2, 8, [f"{avg_volume_share}"], None)
            worksheet.write_column(3, 8, [f"{max_volume_share_row12m["Volume Share_client"]}"], None)
            worksheet.write_column(4, 8,  [f"{max_volume_share_row["Volume Share_client"]}"], None)
            if key in modified_price_p12m:
                modified_price_p12m[key]['End of Week'] = pd.to_datetime(modified_price_p12m[key]['End of Week'], errors='coerce').dt.strftime('%Y/%m/%d')
 
                worksheet.write_column(len(unique_in_df1['End of Week'])+3, 4,modified_price_p12m[key]['End of Week'], None)
        chart1._workbook.update_from_xlsx_blob(xlsx_file.getvalue())
        legend = []
 
        if x >= 0  :
            legend.append(("--- P3M Average",RGBColor(87,85,85)))
        if p12m >= 0 and p12m !=x:  # Separate `if` for this condition
            legend.append(("--- P12M Quarterly Highest",RGBColor(0,160,151)))
        if z >= 0 and z!=p12m:  # Separate `if` for this condition
            legend.append(("--- P3Y Quarterly Highest",RGBColor(192, 0, 0)))
 

#######################################Chart2#################################333
        chart_data2 = XyChartData()
        series_P3Y = chart_data2.add_series('P3Y_ Points')
        series_P12M = chart_data2.add_series('P12M_Points')
# Loop through data to determine matched/unmatched points
        for i in range(len(category)):
            point =(x_values_av[i], y_values[i])
            is_matched = False
            if key in modified_price_p12m:
                if point in zip(
                        modified_price_p12m[key]['Av Price/KG_client'],
                        modified_price_p12m[key]['Volume Share_client']):
                    is_matched = True
 
            if is_matched:
                series_P12M.add_data_point(x_values_av[i], y_values[i])
            else:
                series_P3Y.add_data_point(x_values_av[i], y_values[i])
       
        x_values_P12M = []
        y_values_P12M= []
        for point in series_P12M:
            x_values_P12M.append(point.x)
            y_values_P12M.append(point.y)
        if len(x_values_P12M) > 1 and len(y_values_P12M) > 1 and len(set(x_values_P12M)) > 1:  # Ensure there are enough values to fit
            slope, intercept = np.polyfit(x_values_P12M, y_values_P12M, 1)  # Linear fit (degree 1)
            # print(intercept,slope)
        category_axis = chart2.category_axis
        min_xvalue2 =min(x_values_av)
        max_xvalue2= max(x_values_av)  
 
        ###############P3yQ Line#################
        z=( max_volume_share_row['Volume Share_client']-intercept)/slope
        line_series = chart_data2.add_series('Custom Linep3yQ')
        line_series.add_data_point(0, max_volume_share_row['Volume Share_client'])
        line_series.add_data_point(z,  max_volume_share_row['Volume Share_client'])  
        line2_series = chart_data2.add_series("Custom Line2p3yQ")
        line2_series.add_data_point(z,  max_volume_share_row['Volume Share_client'])
        line2_series.add_data_point(z, 0)
        ###############P12MQ Line#################
        if key in modified_price_p12m :
            p12m=(max_volume_share_row12m['Volume Share_client']-intercept)/slope
            line_series = chart_data2.add_series('Custom Linep12mQ')
            line_series.add_data_point(0, max_volume_share_row12m['Volume Share_client'])
            line_series.add_data_point(p12m,  max_volume_share_row12m['Volume Share_client'])
            line2_series = chart_data2.add_series("Custom Line2p12mQ")
            line2_series.add_data_point(p12m,  max_volume_share_row12m['Volume Share_client'])
            line2_series.add_data_point(p12m, 0)    
 
        ##############P3M Line#################
        if key in categories_dict_P3M and not categories_dict_P3M[key].empty:
            # Calculate the average volume share
            avg_volume_share = categories_dict_P3M[key]["Volume Share_client"].mean()
            x=(avg_volume_share-intercept)/slope
            line_series = chart_data2.add_series('Custom Linep3m')
            line_series.add_data_point(0, avg_volume_share)
            line_series.add_data_point(x, avg_volume_share)
            line2_series = chart_data2.add_series('Custom Line2p3m')
            line2_series.add_data_point(x, avg_volume_share)
            line2_series.add_data_point(x, 0)
           
        x_values_P12M.append(p12m)
        x_values_P12M.append(z)
        x_values_P12M.append(x)    
        ###############Regression line####################
        series_Y_Hat = chart_data2.add_series("Y_Hat")
        # Compute y_hat for each x-value
        for x in x_values_P12M:
            y_hat = slope * x + intercept
            # Add the computed y_hat to series_Y_Hat
            series_Y_Hat.add_data_point(x, y_hat)
   
        chart2.replace_data(chart_data2)
        value_axis = chart2.value_axis
        value_axis.minimum_scale =min_yvalue * 0.8
        value_axis.maximum_scale = max_yvalue * 1.2
        # chart2.category_axis.tick_labels.number_format = '#,##0.00'+ currency
        chart2.category_axis.tick_labels.number_format = '#,##0.00'  if decimals == 2 else '#,##0'
        category_axis = chart2.category_axis
        smallest = min(min_xvalue2, x, p12m, z)
        largest = max(max_xvalue2, x, p12m, z)
        if smallest <0:
            smallest =0
        category_axis.minimum_scale=round(smallest*0.8)
        category_axis.maximum_scale=round(largest*1.2)
       
        currencywithoutspace =currency.strip()  # Remove the leading space
        category_axis.axis_title.text_frame.text = f"Avg Price / Vol ({currencywithoutspace})"  # Set the axis title text  
        category_axis.axis_title.text_frame.paragraphs[0].font.size = Pt(8)
        category_axis.axis_title.text_frame.paragraphs[0].font.bold = False
        category_axis.axis_title.text_frame.paragraphs[0].font.name = 'Nexa Bold'        
        xlsx_file = BytesIO()
        with chart_data2._workbook_writer._open_worksheet(xlsx_file) as (workbook, worksheet):
            chart_data2._workbook_writer._populate_worksheet(workbook, worksheet)
            worksheet.write(0, 4, "End of Week")
            worksheet.write_column(1, 4, unique_in_df1['End of Week'], None)
            worksheet.write(0, 7, "Highest Value")
            worksheet.write_column(2, 7, ["P3M Avg"], None)
            worksheet.write_column(3, 7, ["P12M HighestQ"], None)
            worksheet.write_column(4, 7, ["P3Y HighestQ"], None)
            # Write to worksheet
            worksheet.write_column(2, 8, [f"{avg_volume_share}"], None)
            worksheet.write_column(3, 8, [f"{max_volume_share_row12m["Volume Share_client"]}"], None)
            worksheet.write_column(4, 8,  [f"{max_volume_share_row["Volume Share_client"]}"], None)
            if key in modified_price_p12m:
                modified_price_p12m[key]['End of Week'] = pd.to_datetime(modified_price_p12m[key]['End of Week'], errors='coerce').dt.strftime('%Y/%m/%d')
                worksheet.write_column(len(unique_in_df1['End of Week'])+3, 4,modified_price_p12m[key]['End of Week'], None)
        chart2._workbook.update_from_xlsx_blob(xlsx_file.getvalue())
     
        if x >= 0 and ("--- P3M Average", RGBColor(87, 85, 85)) not in legend:
            legend.insert(0,("--- P3M Average",RGBColor(87,85,85)))
        if p12m >= 0 and p12m != x and ("--- P12M Quarterly Highest",RGBColor(0,160,151)) not in legend:  # Separate `if` for this condition
            legend.insert(1,("--- P12M Quarterly Highest",RGBColor(0,160,151)))
        if z >= 0 and z!=p12m and ("--- P3Y Quarterly Highest",RGBColor(192, 0, 0)) not in legend :  # Separate `if` for this condition
            legend.insert(2,("--- P3Y Quarterly Highest",RGBColor(192, 0, 0)))
        # Access the text frame of the shape
        text_frame = shapes[10].text_frame
        text_frame.clear()  # Clear existing text
        for idx, (text, color) in enumerate(legend):
            # If not the first entry, add a newline
            if idx > 0:
                text_frame.add_paragraph()  # Add a new paragraph
            # Create a new run for this text entry
            p = text_frame.paragraphs[-1]  # Get the last paragraph (which is the current one)
            run = p.add_run()  # Add a run to this paragraph
            run.text = text  # Set the text
            run.font.color.rgb = color  # Set the font color for this run
            run.font.size = Pt(7)  # Set the font size (optional)
            run.font.name = "Nexa Book"  #
       
        slide_num +=1

# Price Point Distribution (Scraped Av Price)

In [ ]:
def pricePointScraped(prs,sectorPD,numOfDuplicates,sec_seg='Sector',position=0):
     """
        Generate slides displaying price point distribution.
 
        Args:
        - prs: PowerPoint presentation object.
        - sectorPD (dict): Dictionary containing DataFrames with price point information for each sector.
        - brandInScope (str): Name of the brand in focus.
        - numOfDuplicates (int): Number of duplicate slides to generate.
        - months (str): Time period for the data.
        - sec_seg (str): The name of the column representing the sector or segment.
        - position (int): Position of the slide in the presentation.
        """
     for slidenum in range(numOfDuplicates):
        dfName=list(sectorPD.keys())[slidenum]
        shapes = prs.slides[slidenum+position].shapes
        titleNumber = get_shape_number(shapes, "Price Point Distribution | Brand | Category | National | P12M")
        headerNumber = get_shape_number(shapes, "Price Point Distribution Analysis by product (Replace with SO WHAT)")
        axistitle=get_shape_number(shapes,"Base Price/Unit ($)")
        currencywithoutspace =currency.strip()  # Remove the leading space
        shapes[axistitle].text=f'Scraped Av. Price/Unit ({currencywithoutspace})'
        shapes[axistitle].text_frame.paragraphs[0].font.size = Pt(8)
        shapes[axistitle].text_frame.paragraphs[0].font.name = 'Nexa Bold'
        shapes[axistitle].text_frame.paragraphs[0].font.color.rgb = RGBColor(87, 85, 85)  # Set color to black
 
        shapes[titleNumber-1].text = data_source # Set data source information
        # Set title for the slide
        shapes[titleNumber].text = "Price Point Distribution | " + list(sectorPD.keys())[slidenum] 
        # shapes[titleNumber].text_frame.paragraphs[0].font.bold = True
        shapes[titleNumber].text_frame.paragraphs[0].font.size = Pt(12)
        shapes[titleNumber].text_frame.paragraphs[0].font.name = 'Nexa (Headings)'    
        tables,charts=createTableAndChart(shapes)
        chart=charts[0].chart
        chart_data = CategoryChartData()
        chart_data.categories = sectorPD[dfName][prodORitem].unique()
        # formatted_prices = sectorPD[dfName]['Base Price/Unit'].apply(lambda x: f"{currency}{round(x, decimals):,.2f}")
        # chart_data.add_series('Base Price/Unit', formatted_prices)
       
        chart_data.add_series('Scraped Av. Price/Unit', round(sectorPD[dfName]['Scraped Av. Price/Unit'].astype(float),decimals))
        chart.replace_data(chart_data)
        chart.has_title = False
 
        for i,series in enumerate(chart.series):
                for j, point in enumerate(series.points):      
                    data_label = point.data_label
                    data_label.has_text_frame = True
                    data_label.text_frame.text = str(round(series.values[j], decimals))
                    data_label.position = XL_LABEL_POSITION.ABOVE  # Set the position above the point
 
        value_axis = chart.value_axis
        value_axis.auto_axis = True
        value_axis.minimum_scale = None
        value_axis.major_unit = None
        value_axis.minor_unit = None
        value_axis.axis_title.text_frame.text=''
        value_axis.tick_labels.number_format = "#,##0.00" if decimals ==2 else "#,##0"
 
        # Table #
        table=tables[0].table
 
        num_columns_to_remove = (len(table.columns) - sectorPD[dfName].shape[0]) - 1  # Specify the number of rows to remove from the end
        table_width = Inches(8.89)  # Specify the desired table height
        # table=col_cell_remove(table,num_columns_to_remove,table_width,sectorPD[dfName])
        table=col_cell_remove(table,num_columns_to_remove)
        # print(num_columns_to_remove)
       
       
        total_col_width = table_width - table.columns[0].width
        num_columns = len(table.columns) - 1  # Exclude the first row
 
        if num_columns > 0:
            cell_width = total_col_width / num_columns
            for col in range(1, table.columns.__len__()):
                table.columns[col].width = int(cell_width)
 
        # mergedCellDf = sectorPD[dfName][sec_seg].value_counts().reset_index()
        mergedCellDf = sectorPD[dfName].groupby(sec_seg).size().reset_index(name='count')
        # print(dfName, mergedCellDf)

        # print(dfName, sectorPD[dfName][sec_seg])
        # mergedCellDf = mergedCellDf.rename(columns = {"index":f'{sec_seg}'})
        # mergedCellDf=mergedCellDf.merge(sectorPD[dfName][[sec_seg,'Sort_Value']].drop_duplicates()).sort_values(['Sort_Value'])
 
        mergedCellDf['mergedCell']=mergedCellDf['count'].cumsum()
        count=1
        # print(dfName,mergedCellDf['mergedCell'].unique())
        for mergedCell in mergedCellDf['mergedCell'].unique():
            table.cell(0,count).merge(table.cell(0,mergedCell))
            count=mergedCell+1
 
        table.height = Inches(1.54)
       
        for i, row in enumerate(table.rows):
            for j, cell in enumerate(row.cells):
                if j == 0:
                    if i == 3:
                        cell.text=f"Scraped Av. Price/Vol ({currencywithoutspace})"
                        cell.text_frame.paragraphs[0].alignment = PP_ALIGN.CENTER
                        cell.text_frame.paragraphs[0].runs[0].font.size = Pt(6)
                        cell.text_frame.paragraphs[0].runs[0].font.bold = False
                        cell.text_frame.paragraphs[0].font.name = 'Nexa (Headings)'
                    continue
               
                if i==0:
                    cell.text=sectorPD[dfName][sec_seg].iloc[j-1]
                    cell.text_frame.paragraphs[0].alignment = PP_ALIGN.CENTER
                    cell.text_frame.paragraphs[0].runs[0].font.size = Pt(8)
                    cell.text_frame.paragraphs[0].runs[0].font.bold = True
                    cell.text_frame.paragraphs[0].font.name = 'Nexa Bold'
                elif (i==1):
                    cell.text=sectorPD[dfName][f"{prodORitem}"].iloc[j-1]
                    cell.text_frame.paragraphs[0].alignment = PP_ALIGN.CENTER
                    cell.text_frame.paragraphs[0].runs[0].font.size = Pt(6)
                    cell.text_frame.paragraphs[0].runs[0].font.bold = True
                    cell.text_frame.paragraphs[0].font.name = 'Nexa Bold'
                elif (i==2):
                    cell.text=sectorPD[dfName]['Total Size'].iloc[j-1]
                elif (i==3):
                    cell.text=str(round(sectorPD[dfName]['Scraped Av. Price/KG'].astype(float).iloc[j-1],decimals))
                       
                if cell.text!='' and i!=0 and i!=1:
                    cell.text_frame.paragraphs[0].alignment = PP_ALIGN.CENTER
                    cell.text_frame.paragraphs[0].runs[0].font.size = Pt(7)
                    cell.text_frame.paragraphs[0].runs[0].font.bold = False
                    cell.text_frame.paragraphs[0].font.name = 'Nexa Book'

# Cleaning Functions

In [ ]:
def dropemptydf(dic):
    keys_to_remove = [k for k, v in dic.items() if v.empty]
    for k in keys_to_remove:
        del dic[k]
        
def remove_others_from_dic(sorted_dic, col):
    for k in sorted_dic.keys():
        sorted_dic[k] = sorted_dic[k][~sorted_dic[k][col].str.contains("Others", case=False, na=False)]
#####PricepositionSlideClean      
def PricePositioningClean(dic, col=" "):
    modified_price_positioning = {}
    for l in dic.keys():
        df = dic[l].copy()  # Create a copy of the DataFrame
        # df = DetectHeader(df)  # Detect and clean headers
        df['Value Share'] = df['Value Share'].astype(float)
        df = df[df[col] != "Grand Total"]
        df=df[~df[col].str.contains("Others")]
        # if not df.empty:
        modified_price_positioning[l] = df
        
    sorted_dic_position = dfSort(modified_price_positioning, client_brands, col, num=9, salesCol='Value Share')
    sorted_dic_table = dfSort(modified_price_positioning, client_brands, col, num=6, salesCol='Value Share')
    remove_others_from_dic(sorted_dic_position, col)
    remove_others_from_dic(sorted_dic_table, col)
    dropemptydf(sorted_dic_position)
    dropemptydf(sorted_dic_table)

    return sorted_dic_position ,sorted_dic_table # Return the sorted dictionary
#####Sector/Segment Leadership TableClean
def sharGrowthDfclean(dic, totdic, valid_list):
    sharGrowthDf = {}
    keys_to_check = (
        ["NATIONAL"] + regions_RET + channels_RET + market_RET +
        regions_CHAN + channels_CHAN + market_CHAN +
        regions_CUST + channels_CUST + market_CUST
    )
    
    for key in keys_to_check:
        matched = [k.split(' | ')[0] for k in dic.keys() 
                   if k.split(' | ')[1] == key and k.split(' | ')[0] in valid_list]

        if matched and key in totdic:
            df = totdic[key].copy()
            order = list(df[df.columns[0]].dropna().astype(str))  # Ensure all values are strings for matching
            # Filter and sort matched by order
            matched_sorted = [x for x in order if x in matched]
            sharGrowthDf[key] = [matched_sorted[i:i+6] for i in range(0, len(matched_sorted), 6)]

    return sharGrowthDf

####Sector/Segment Leadership Analysis clean
def leadershipanalysisclean(inputdic,outputdic,parent="sector",total=False):
    for key in inputdic.keys():
        tot1 = inputdic[key].copy()
        # tot1=DetectHeader(tot1)
        tot1[parent] = tot1[parent].ffill()
        tot1["Gross Margin %"] = tot1["Gross Margin %"].fillna(0)
        tot1 =tot1[~tot1[parent].str.contains("Total",na=False)]
        if total==True:
            tot1=tot1[tot1['Value Share']>=0.01]
            tot1 = tot1.sort_values(by='Value Share', ascending=False)
            top_sectors = tot1.head(10)
            outputdic[key] = top_sectors
        else:
            outputdic[key] = tot1         
            
####Sector/Segment BrandLeadershipsclean      
def totalleadership(indictionary, outdictionary, Inscope="",total=False):
    for k in indictionary.keys():
        t = indictionary[k].copy()
        # t=DetectHeader(t)
        t["Gross Margin %"] = t["Gross Margin %"].fillna(0)
        t =t[~t[Inscope].str.contains("Grand Total",na=False)]
        if total==True:
            t=t[t['Value Share']>=0.01]
            t = t.sort_values(by='Value Share', ascending=False)
            top_sectors = t.head(5)
            outdictionary[k] = top_sectors
        else:
            outdictionary[k] = t       
            
def wobcleaning(inputdic,slideby=""):
    outputdic={}
    for k in inputdic.keys():
        c=inputdic[k].copy()
        # c=DetectHeader(c)
        c=c[~c[slideby].str.contains("Grand Total",na=False)]
        c = c.sort_values("Value Share", ascending=False)
        outputdic[k]=c
    return outputdic                    

In [ ]:
######Shelf price /vol and Avg price /vol comparisonClean
def shelfallbrandsclean (inputalldic,outputalldic,inputtopdic,outputtopdic,Inscope="Sector",bymanuf=""):
    for k in inputalldic.keys():
        t = inputalldic[k].copy()
        # print(k,inputalldic)
        # t=DetectHeader(t)
        t[Inscope] = t[Inscope].ffill()
        t = t[t[Inscope].str.contains('Total')]
        t = t.sort_values('Value Share', ascending=False)
        t[Inscope] = t[Inscope].str.replace(' Total', '')
        t['Sort_Value'] = range(1,t.shape[0]+1)
        t['Sort_Value'] = t['Sort_Value'] * -1
        outputalldic[k] = t
        
    for k in inputtopdic.keys():
        b = inputtopdic[k].copy()
        # b = DetectHeader(b)
        b[Inscope] = b[Inscope].ffill()
        b=b.fillna(0)
        if bymanuf==f'{ManufOrTopC}':
            cop = b[~b[f'{ManufOrTopC}'].isin(client_manuf)]
        
            cop = cop[~cop[Inscope].str.contains('Total')]
            cop_tot = cop[[f'{ManufOrTopC}', 'Value Share']].groupby(f'{ManufOrTopC}', group_keys=False).sum().reset_index()
            cop_tot = cop_tot.sort_values('Value Share', ascending=False)
            cop_tot = cop_tot[~cop_tot[f'{ManufOrTopC}'].str.contains('Other')]
            top_manuf = cop_tot[f'{ManufOrTopC}'][:3].tolist()

            for c in client_manuf:
                if c not in b[f'{ManufOrTopC}'].values:
                    continue

                final = b[b[f'{ManufOrTopC}'].isin([c] + top_manuf)]
            
                final = final.merge(outputalldic[k][[Inscope, 'Sort_Value']]).sort_values(by=['Sort_Value', 'Value Share'], ascending=False)
                outputtopdic[c + ' | ' + k] = final

        else: 
            cop = b[~b[f"{BrandOrTopB}"].isin(client_brands)]
            cop = cop[~cop[Inscope].str.contains('Total')]
            cop_tot = cop[[f"{BrandOrTopB}", 'Value Share']].groupby(f"{BrandOrTopB}", group_keys=False).sum().reset_index()
            cop_tot = cop_tot.sort_values('Value Share', ascending=False)
            cop_tot = cop_tot[~cop_tot[f"{BrandOrTopB}"].str.contains('Other')]
            top_brands = cop_tot[f"{BrandOrTopB}"][:3].tolist()

            for c in client_brands:
                if c not in b[f"{BrandOrTopB}"].values:
                    continue

                final = b[b[f"{BrandOrTopB}"].isin([c] + top_brands)]
                # print(k)
                final = final.merge(outputalldic[k][[Inscope, 'Sort_Value']]).sort_values(by=['Sort_Value', 'Value Share'], ascending=False)
                outputtopdic[c + ' | ' + k] = final
            
        for key in outputtopdic.keys():
            outputtopdic[key]['Av Price/KG'] = pd.to_numeric(outputtopdic[key]['Av Price/KG'], errors='coerce')
            outputtopdic[key]['Top Comp Av Price/KG'] = (
                outputtopdic[key]
                .loc[outputtopdic[key]['Value Share'] == outputtopdic[key].groupby(Inscope)['Value Share'].transform('max'), 'Av Price/KG']
                .groupby(outputtopdic[key][Inscope])
                .transform('max')
            )
            outputtopdic[key]['Top Comp Av Price/KG']=outputtopdic[key]['Top Comp Av Price/KG'].ffill()
            outputtopdic[key]['Ix_Av Price/KG'] = (
                outputtopdic[key].groupby(Inscope)['Av Price/KG'].transform(
                    lambda x: ((x / outputtopdic[key]['Top Comp Av Price/KG']) * 100)
                    )
            )
            outputtopdic[key]['Ix_Av Price/KG'] = (
                outputtopdic[key]['Ix_Av Price/KG']
                .replace([np.inf, -np.inf], np.nan)  # handle inf
                .fillna(0)                           # replace NaN
                .round()
                .astype(int)
            )
            outputtopdic[key]['Ix_Av Price/KG'] = "IX " + outputtopdic[key]['Ix_Av Price/KG'].astype(str)

            outputtopdic[key]["Base Price/KG"] = pd.to_numeric(outputtopdic[key]["Base Price/KG"], errors='coerce')
            # Calculate the Ix within each sector
            outputtopdic[key]['Top Comp Base Price/KG'] = (
            outputtopdic[key]
            .loc[outputtopdic[key]['Value Share'] == outputtopdic[key].groupby(Inscope)['Value Share'].transform('max'), 'Base Price/KG']
            .groupby(outputtopdic[key][Inscope])
            .transform('max')
            )
            outputtopdic[key]['Top Comp Base Price/KG']=outputtopdic[key]['Top Comp Base Price/KG'].ffill()

            outputtopdic[key]['Ix_Base Price/KG'] = (
            outputtopdic[key].groupby(Inscope)['Base Price/KG'].transform(
                lambda x: ((x / outputtopdic[key]['Top Comp Base Price/KG']) * 100)
            )
            )
            outputtopdic[key]['Ix_Base Price/KG'] = (
                outputtopdic[key]['Ix_Base Price/KG']
                .replace([np.inf, -np.inf], np.nan)  # handle inf
                .fillna(0)                           # replace NaN
                .round()
                .astype(int)
            )
            outputtopdic[key]['Ix_Base Price/KG'] = "IX " + outputtopdic[key]['Ix_Base Price/KG'].astype(str)

In [ ]:
#### Shelf Price & AVG PRICE FUNCTION
def shelfPriceCleaning(total_brands, top_brands, sec_seg='Sector',slide_by=[],brandORmanuf=''):
    all_brands = {}

    for top_key in top_brands.keys():
        common_part = top_key.split(" | ",1)[1]  # Extract the common part from the top_brands key
        total_key = [key for key in total_brands.keys() if common_part in key][0]  # Find the corresponding total_brands key
        
        if 'Sort_Value' in total_brands[total_key].columns:
            total_brands[total_key] = total_brands[total_key].drop(columns=['Sort_Value'])
        total_brands[total_key] = total_brands[total_key].sort_values(['Value Share'], ascending=False)
        total_brands[total_key]['Sort_Value'] = range(1, total_brands[total_key].shape[0] + 1)

        total_brands[total_key][brandORmanuf] = 'Total'
        total_brands[total_key][sec_seg] = total_brands[total_key][sec_seg].str.replace('Total', '').str.strip()
        
        all_brands[top_key] = pd.concat([total_brands[total_key], top_brands[top_key]])
        unique_brands = all_brands[top_key][brandORmanuf].unique()
        missing_brand_sector = {}
        
        for sector in all_brands[top_key][sec_seg].unique():
            missingBrands = list(set(unique_brands) - set(all_brands[top_key][all_brands[top_key][sec_seg] == sector][brandORmanuf].unique()))
            if missingBrands:
                missing_brand_sector[sector] = missingBrands

        missing_brand_sector = pd.DataFrame({sec_seg: missing_brand_sector.keys(), brandORmanuf: missing_brand_sector.values()}).explode(brandORmanuf)
        all_brands[top_key] = pd.concat([all_brands[top_key], missing_brand_sector]).sort_values(by=[sec_seg]).replace(np.nan, None)
        all_brands[top_key]['Value Share'] = all_brands[top_key]['Value Share'].astype(float)
        total_brands[total_key]['Sort_Value'] = total_brands[total_key]['Sort_Value'].astype(int)
        total_brands[total_key] = total_brands[total_key][total_brands[total_key][sec_seg].isin(slide_by)]
        all_brands[top_key] = all_brands[top_key].drop(columns='Sort_Value').merge(total_brands[total_key][[sec_seg, 'Sort_Value']]).sort_values(['Sort_Value', 'Value Share'], ascending=[True, False])
        all_brands[top_key] = all_brands[top_key][all_brands[top_key][sec_seg].isin(slide_by)]
        
    return all_brands

In [ ]:
def cleaningPricePointScraped(dfs,Eandfs,Inscope):
    output={}
    for k in dfs.keys():
        t = dfs[k].copy()
        t=DetectHeader(t)
        endf=Eandfs[k].copy()
        endf=DetectHeader(endf)
        
        # t = t[(t['Total Size'].notna()) | (t[Inscope].str.contains("Total"))]
        t["End of Week"]=t["End of Week"].ffill()
        t[Inscope] = t[Inscope].ffill()
        t["Ean"] = t["Ean"].ffill()
        t["SKU"] = t["SKU"].ffill()
        
        dfTotal = t[t[Inscope].str.contains('Total')]
        # dfTotal['Sort_Value']=range(1,dfTotal.shape[0]+1)
        # dfTotal[Inscope]=dfTotal[Inscope].str.replace(' Total','')
        t = t[~t[Inscope].str.contains('Total')]
        t=t[~t["End of Week"].str.contains('Total')]  
        t = t[~t[f'{prodORitem}'].fillna('').str.contains('Total')]
        t=t[~t['Ean'].str.contains('Total')] 
        
        # t=t.dropna(subset=['Scraped Av. Price/Unit', 'Scraped Av. Price/KG'])
        
        t['End of Week'] = pd.to_datetime(t['End of Week'], errors='coerce')
        # t.drop(columns=['End of Week'], inplace=True)
        # dfTotal['Sort_Value']=range(1,dfTotal.shape[0]+1)
        # dfTotal[Inscope]=dfTotal[Inscope].str.replace(' Total','')
        # t = t[~t[Inscope].str.contains('Total')].head(15)
        # t=t.merge(dfTotal[[Inscope,'Sort_Value']]).sort_values(['Sort_Value','Scraped Av. Price/KG'],ascending=[True,False])
        if not t.empty and k in Eandfs:
            t = t.merge(endf, on='Ean', how='left')
            t['End of Week'] = pd.to_datetime(t['End of Week'], errors='coerce')
            latest_eow = t['End of Week'].max().date()
            t = t[t['End of Week'].dt.date == latest_eow]
            t = t.sort_values('Value Sales',ascending=False).head(15)

            # print(latest_eow)

            output[k] = t
             # missing_values = sectorPD[key][['Base Price/Unit', 'Base Price/KG']].isna().any()
    return output 

In [ ]:
###########Price Point Slides clean 
def cleaningPricePoint(dfs,Inscope):
    output={}
    for k in dfs.keys():
        t = dfs[k].copy()
        # t=DetectHeader(t)
        t = t[(t['Total Size'].notna()) | (t[Inscope].str.contains("Total"))]
        t[Inscope] = t[Inscope].ffill()
        t['Value Sales'] = t['Value Sales'].fillna(0)
        t = t[t['Value Sales']>1000]
        t = t.sort_values('Value Sales',ascending = False)
        dfTotal = t[t[Inscope].str.contains('Total')]
        dfTotal=dfTotal.sort_values(['Value Sales'],ascending=False)
        dfTotal['Sort_Value']=range(1,dfTotal.shape[0]+1)
        dfTotal[Inscope]=dfTotal[Inscope].str.replace(' Total','')
        t = t[~t[Inscope].str.contains('Total')].head(15)
        t=t.merge(dfTotal[[Inscope,'Sort_Value']]).sort_values(['Sort_Value','Value Sales'],ascending=[True,False])
        t=t.dropna(subset=['Base Price/Unit', 'Base Price/KG'])
        if not t.empty:
            output[k] = t             # missing_values = sectorPD[key][['Base Price/Unit', 'Base Price/KG']].isna().any()
    return output           
def clean_P12M(data,slideby=""):
    modified_price_point_by_brands_items_P12M = {}
    final ={}
    for key in data.keys():
        df = data[key]
        # df=DetectHeader(df)
        df[slideby] = df[slideby].ffill()
        df = df[~df[slideby].str.contains('Total')]
        df = df.reset_index(drop =True)
        if df.shape[0] >0:
            modified_price_point_by_brands_items_P12M[key] = df
    for key, df in modified_price_point_by_brands_items_P12M.items():
        for client in df[slideby].unique():
            if client in df[slideby].unique():
                new_key = key + ' | ' + client
                if df.shape[0] > 0:
                    final[new_key] = df
                    
    return final
def clean_P3M(data,slideby=""):
    cleaned_data = {}
    final ={}
    for key in data.keys():
        df = data[key]
        # df=DetectHeader(df)
        df[slideby] = df[slideby].ffill()
        df=df[df["Value Sales"]>1000]
        if df.shape[0] >0:
            cleaned_data[key] = df
            
    for key, df in cleaned_data.items():
        total_entries = df[(df[slideby].str.contains(' Total')) & ~(df[slideby].isin(['Grand Total',"All Others Total"])) & ~(df[slideby].isin([i+' Total' for i in client_brands]))]
        total_entries['Value Share'] = total_entries['Value Share'].astype(float)
        total_entries = total_entries.nlargest(3,columns="Value Share")    
        comp_lis = list(total_entries[slideby].str.replace(" Total",''))
        if slideby==f"{BrandOrTopB}":
            for client in client_brands:
                if client in df[f"{BrandOrTopB}"].unique():
                    df = df[~df[f"{BrandOrTopB}"].isin(['Grand Total',"All Others Total"])]
                    df = df[~df[f"{BrandOrTopB}"].str.contains('Total')]
                    df[f'{prodORitem}'] = df[f'{prodORitem}'].astype(str)

            
                    df = df[~df[f'{prodORitem}'].str.contains('Total')]
                    
                    df['Base Price/Unit'] = df['Base Price/Unit'].fillna(0)
                    df['Base Price/KG'] = df['Base Price/KG'].fillna(0)
                    df = df.reset_index(drop =True)
                    
                    comp_sorted = df[
                        (~df[f"{BrandOrTopB}"].isin(client_brands)) & 
                        (df[f"{BrandOrTopB}"].isin(comp_lis))
                    ].sort_values(by="Value Share", ascending=False)
                    top_5_comp = comp_sorted.groupby(f"{BrandOrTopB}", group_keys=False).apply(lambda x: x.sort_values(by='Value Share', ascending=False).head(5)).reset_index(drop = True)
                    
                    client_sorted = df[df[f"{BrandOrTopB}"]==client].sort_values(by ="Value Share", ascending = False)
                    top_5_client = client_sorted.reset_index(drop=True).head(5)
                    merged = pd.concat([top_5_client, top_5_comp], ignore_index=True)
                    new_key = key + ' | ' + client
                    if merged.shape[0] > 0:
                            final[new_key] = merged
        else:
            for manuf in client_manuf:
                if manuf in df[slideby].unique():
                    df = df[~df[slideby].isin(['Grand Total',"All Others Total"])]
                    df = df[~df[slideby].str.contains('Total')]
                    df[f'{prodORitem}'] = df[f'{prodORitem}'].astype(str)

                    df = df[~df[f'{prodORitem}'].str.contains('Total')]
                    
                    df['Base Price/Unit'] = df['Base Price/Unit'].fillna(0)
                    df['Base Price/KG'] = df['Base Price/KG'].fillna(0)
                    df = df.reset_index(drop =True)
                    
                    comp_sorted = df[
                        (~df[slideby].isin(client_brands)) & 
                        (df[slideby].isin(comp_lis))
                    ].sort_values(by="Value Share", ascending=False)
                    top_5_comp = comp_sorted.groupby(slideby, group_keys=False).apply(lambda x: x.sort_values(by='Value Share', ascending=False).head(5)).reset_index(drop = True)
                    
                    client_sorted = df[df[slideby]==manuf].sort_values(by ="Value Share", ascending = False)
                    top_5_client = client_sorted.reset_index(drop=True).head(5)
                    merged = pd.concat([top_5_client, top_5_comp], ignore_index=True)
                    new_key = key + ' | ' + manuf
                    if merged.shape[0] > 0:
                            final[new_key] = merged

    return final


In [ ]:
###### Cleaning Price Distribution By Brand
def brandPriceDistributionCleaning(dfs,brandInScope=['Kerrygold']):
    brandPriceDistribution={}
    for key in dfs.keys():
        brandPriceDistributionCopy = dfs[key].copy()
        # brandPriceDistributionCopy=DetectHeader(brandPriceDistributionCopy)
        brandPriceDistributionCopy[f"{BrandOrTopB}"]=brandPriceDistributionCopy[f"{BrandOrTopB}"].ffill()
        brandPriceDistributionCopy['Value Share']=brandPriceDistributionCopy['Value Share'].astype(float)
        clientDfTotal = brandPriceDistributionCopy[brandPriceDistributionCopy[f"{BrandOrTopB}"].isin(i + ' Total' for i in brandInScope)].sort_values(['Value Share'],ascending=False)
        clientDfTotal[f"{BrandOrTopB}"] = clientDfTotal[f"{BrandOrTopB}"].str.replace(' Total','')
        clientDf = brandPriceDistributionCopy[brandPriceDistributionCopy[f"{BrandOrTopB}"].isin(brandInScope)]
        brandPriceDistributionCopy=brandPriceDistributionCopy[brandPriceDistributionCopy['Value Share']>=0.005]
        if len(brandInScope)==0:
           brandPriceDistributionCopy=brandPriceDistributionCopy[brandPriceDistributionCopy['Value Share']>=0.01]
        brandPriceDistributionCopy= pd.concat([brandPriceDistributionCopy, clientDf]).drop_duplicates()        
        # Sort the brands on the Total Value Share 
        dfTotal=brandPriceDistributionCopy[brandPriceDistributionCopy[f"{BrandOrTopB}"].str.contains('Total')].sort_values(['Value Share'],ascending=False)
        dfTotal[f"{BrandOrTopB}"]=dfTotal[f"{BrandOrTopB}"].str.replace(' Total','')
        dfTotal=dfTotal[~dfTotal[f"{BrandOrTopB}"].isin(['Other'] + brandInScope)] # Drop The brand named 'Other'
        num_res = 10 - clientDfTotal[f"{BrandOrTopB}"].nunique()
        dfTotal = pd.concat([dfTotal.iloc[:num_res],clientDfTotal]).sort_values(['Value Share'], ascending=False)
        dfTotal['Sort_Value']=range(1,dfTotal.shape[0]+1)
        brandPriceDistributionCopy=brandPriceDistributionCopy[~brandPriceDistributionCopy[f"{BrandOrTopB}"].str.contains(' Total')]#drop total after sort
        brandPriceDistributionCopy=brandPriceDistributionCopy.merge(dfTotal[[f"{BrandOrTopB}",'Sort_Value']]).sort_values('Sort_Value')        
        # filling the missing pack size for each brand for drawing the chart
        unique_size=brandPriceDistributionCopy['Pack Size'].unique()
        missing_brand_size={}
        for brand in brandPriceDistributionCopy[f"{BrandOrTopB}"]:
            missingSizes=list(set(unique_size)-set(brandPriceDistributionCopy[brandPriceDistributionCopy[f"{BrandOrTopB}"]==brand]['Pack Size'].unique()))
            if missingSizes:
                missing_brand_size[brand]=missingSizes
        missing_brand_size=pd.DataFrame({f"{BrandOrTopB}":missing_brand_size.keys(),'Pack Size':missing_brand_size.values()}).explode('Pack Size')
        brandPriceDistributionCopy=pd.concat([brandPriceDistributionCopy,missing_brand_size])
        brandPriceDistributionCopy['Pack Size No']=brandPriceDistributionCopy['Pack Size'].str.replace('GR','').str.replace('OZ','')
        brandPriceDistributionCopy['Is_Client'] = brandPriceDistributionCopy[f"{BrandOrTopB}"].isin(client_brands)
        brandPriceDistributionCopy=brandPriceDistributionCopy.drop(columns='Sort_Value').merge(dfTotal[[f"{BrandOrTopB}",'Sort_Value']]).sort_values(['Is_Client','Sort_Value','Pack Size No'],ascending=[False,True,True])
        brandPriceDistributionCopy=brandPriceDistributionCopy.fillna(0)
        brandPriceDistributionCopy= brandPriceDistributionCopy.reset_index() 
        brandPriceDistribution[key]=brandPriceDistributionCopy
    return brandPriceDistribution


In [ ]:
###### Cleaning Price Distribution By Brand by Sec/Seg/SubSeg/SUbCat
def segmentPriceDistributionCleaning(brandPriceDistributiontest,brandInScope=['Kerrygold'],sec_seg='Segment',sec_seg_InScope = []):
    brandPriceDistributionfinal={}
    for key in brandPriceDistributiontest.keys():
        brandPriceDistribution=brandPriceDistributiontest[key].copy()
        # brandPriceDistribution[key]=DetectHeader(brandPriceDistribution[key])
        brandPriceDistribution[f"{BrandOrTopB}"] = brandPriceDistribution[f"{BrandOrTopB}"].ffill()
        brandPriceDistribution[sec_seg] = np.where(brandPriceDistribution[f"{BrandOrTopB}"].str.contains(' Total'), brandPriceDistribution[f"{BrandOrTopB}"], brandPriceDistribution[sec_seg])
        brandPriceDistribution[sec_seg] = brandPriceDistribution[sec_seg].ffill()
        brandPriceDistribution[f"{BrandOrTopB}"]=brandPriceDistribution[f"{BrandOrTopB}"].ffill()
        # brandPriceDistribution['Value Share']=brandPriceDistribution['Value Share'].astype(float)
        clientDf = brandPriceDistribution[brandPriceDistribution[f"{BrandOrTopB}"].isin(i + ' Total' for i in brandInScope)]
        clientDf = clientDf.sort_values("Value Share", ascending=False)
        clientDf[f"{BrandOrTopB}"] = clientDf[f"{BrandOrTopB}"].str.replace(" Total", "")
        # Exclude Brands with < 0.5% Market Share
        brandPriceDistribution=brandPriceDistribution[(brandPriceDistribution['Value Share']>=.005)]
        # Sort the brands on the Total Value Share
        dfTotal=brandPriceDistribution[brandPriceDistribution[f"{BrandOrTopB}"].str.contains('Total')].sort_values(['Value Share'],ascending=False)
        dfTotal[f"{BrandOrTopB}"]=dfTotal[f"{BrandOrTopB}"].str.replace(' Total','')
        dfTotal=dfTotal[dfTotal[f"{BrandOrTopB}"]!='Other'] # Drop The brand named 'Other'
        dfTotal=dfTotal.iloc[:5,:]
        missingBrand = list(set(clientDf[f"{BrandOrTopB}"].unique())-set(dfTotal[f"{BrandOrTopB}"].unique()))
        if missingBrand:
            existBrand = dfTotal[dfTotal[f"{BrandOrTopB}"].isin(brandInScope)]
            topBrandsWithoutClient=dfTotal[~dfTotal[f"{BrandOrTopB}"].isin(brandInScope)]
            dfTotal = pd.concat([topBrandsWithoutClient.iloc[:topBrandsWithoutClient.shape[0] - len(missingBrand)],clientDf[clientDf[f"{BrandOrTopB}"].isin(missingBrand)],existBrand]).sort_values(['Value Share'],ascending=False)
        else:
            pass
        dfTotal['Sort_Value_brands'] = range(1,dfTotal.shape[0]+1)
        brandPriceDistribution = brandPriceDistribution[~brandPriceDistribution[f"{BrandOrTopB}"].str.contains(' Total')]#drop total after sort
        dfTotalSegment = brandPriceDistribution[brandPriceDistribution[sec_seg].str.contains('Total')]#.sort_values(['Value Share'],ascending=False)
        dfTotalSegment = dfTotalSegment[dfTotalSegment[f"{BrandOrTopB}"].isin(dfTotal[f"{BrandOrTopB}"].unique())]
        segmentDf=[]
        for brand in dfTotalSegment[f"{BrandOrTopB}"].unique():
            segmentDfPerBrand=dfTotalSegment[dfTotalSegment[f"{BrandOrTopB}"]==brand].sort_values(['Value Share'],ascending=False)
            segmentDfPerBrand['Sort_Value_'+sec_seg]=range(1,segmentDfPerBrand.shape[0]+1)
            segmentDfPerBrand[sec_seg]=segmentDfPerBrand[sec_seg].str.replace(' Total','')
            unique_size=brandPriceDistribution['Pack Size'].unique()
            eachBrandDf=brandPriceDistribution[brandPriceDistribution[f"{BrandOrTopB}"]==brand]
            eachBrandDf=eachBrandDf[~eachBrandDf[sec_seg].str.contains('Total')]
            missing_brand_size={}
            for segment in eachBrandDf[sec_seg].unique():
                missingSizes=list(set(unique_size)-set(eachBrandDf[eachBrandDf[sec_seg]==segment]['Pack Size'].unique()))
                missing_brand_size[segment] = missingSizes
            missing_brand_size=pd.DataFrame({f"{BrandOrTopB}":brand,sec_seg:missing_brand_size.keys(),'Pack Size':missing_brand_size.values()}).explode('Pack Size')
            brandPriceDistribution=pd.concat([brandPriceDistribution,missing_brand_size])#.sort_values(by=[sec_seg]).replace(np.nan,'0')
            brandPriceDistribution['Pack Size No']=brandPriceDistribution['Pack Size'].str.replace('GR','').str.replace('OZ','')
            segmentDf.append(segmentDfPerBrand)
        if segmentDf:    
            segmentDf=pd.concat(segmentDf)
            brandPriceDistribution=brandPriceDistribution.merge(dfTotal[[f"{BrandOrTopB}",'Sort_Value_brands']])#.sort_values(['Sort_Value_brands','Sort_Value_Segment','Pack Size No'])
            brandPriceDistribution=brandPriceDistribution.merge(segmentDf[[f"{BrandOrTopB}",sec_seg,'Sort_Value_'+sec_seg]],on=[f"{BrandOrTopB}",sec_seg]).sort_values(['Sort_Value_brands','Sort_Value_'+sec_seg,'Pack Size No'])
        brandPriceDistribution=brandPriceDistribution[brandPriceDistribution['Pack Size'].notna()]
        brandPriceDistribution=brandPriceDistribution.fillna(0)
        brandPriceDistributionfinal[key]=brandPriceDistribution[brandPriceDistribution[sec_seg].isin(sec_seg_InScope)]
    return brandPriceDistributionfinal

In [ ]:
#### Correlation slide clean
def clean_data(price_correlation_data,p3y=False):
    cleaned_data = {}
    final ={}
    for key in price_correlation_data:
        df = price_correlation_data[key].copy()
        # df=DetectHeader(df)
        df[f"{BrandOrTopB}"] = df[f"{BrandOrTopB}"].fillna(method='ffill')
        df = df[~df[f"{BrandOrTopB}"].str.contains("All Others", na=False)]
        df.fillna(0,inplace = True)
        if p3y==True:
            df['QuarterStart'] = df['QuarterStart'].astype(str)
            # Apply the filter to exclude rows where 'QuarterStart' contains 'Total'
            df = df[~df['QuarterStart'].str.contains('Total', case=False, na=False)]
        if df.shape[0] > 0: 
            cleaned_data[key] = df
    for key, df in cleaned_data.items():
        for client in client_brands:
            if client in df[f"{BrandOrTopB}"].unique():
                total_sorted = df[df[f"{BrandOrTopB}"].str.contains('total', case=False)]
                total_sorted[f"{BrandOrTopB}"] = total_sorted[f"{BrandOrTopB}"].str.replace(' Total', '', regex=False)
                total_sorted = total_sorted[~total_sorted[f"{BrandOrTopB}"].isin(client_brands+['Grand'])].sort_values(by ="Value Share", ascending = False)
                top_3 = total_sorted.reset_index(drop=True).head(3)
                top_3[f"{BrandOrTopB}"] = top_3[f"{BrandOrTopB}"].str.replace(' Total', '', regex=False)
                top_3_brands = top_3[f"{BrandOrTopB}"]
                for i in top_3_brands:
                    client_df = df[df[f"{BrandOrTopB}"]== client].reset_index(drop= True)
                    comp_df = df[df[f"{BrandOrTopB}"]== i].reset_index(drop= True)
                    merged_df = pd.merge(client_df, comp_df, on='End of Week', suffixes=('_client', '_comp'))
                    merged_df['Av Price/KG_comp'] = merged_df['Av Price/KG_comp'].replace(0,np.nan)
                    merged_df['Price lx vs PL'] = merged_df['Av Price/KG_client'] / merged_df['Av Price/KG_comp']
                    new_key = key + ' | ' + client + ' | ' + i
                    if merged_df.shape[0] > 0:
                        final[new_key] = merged_df
    
    return final
    

In [ ]:
def compute_r_squared(modified_price, modified_price_p12m):
    r_squared_dict = {}
    filtered_modified_price = {}
 
    for key, df in modified_price.items():
        df.fillna(0, inplace=True)
        r2chart1, r2chart2 = None, None
 
        # --- Chart 1 ---
        if 'Price lx vs PL' in df.columns and 'Volume Share_client' in df.columns:
            df12m = modified_price_p12m.get(key)
            if df12m is not None:
                df12m.fillna(0, inplace=True)
                x_vals1 = df12m['Price lx vs PL'].tolist()
                y_vals1 = df12m['Volume Share_client'].tolist()
 
                if len(x_vals1) > 1 and len(y_vals1) > 1:
                    slope, intercept = np.polyfit(x_vals1, y_vals1, 1)
                    y_hat = [slope * x + intercept for x in x_vals1]
                    ss_res = sum((y - y_h) ** 2 for y, y_h in zip(y_vals1, y_hat))
                    ss_tot = sum((y - np.mean(y_vals1)) ** 2 for y in y_vals1)
                    r2chart1 = 1 - ss_res / ss_tot if ss_tot != 0 else 0
 
        # --- Chart 2 ---
        if key in modified_price_p12m:
            df12m = modified_price_p12m[key]
            df12m.fillna(0, inplace=True)
 
            if 'Av Price/KG_client' in df12m.columns and 'Volume Share_client' in df12m.columns:
                x_vals2 = df12m['Av Price/KG_client'].tolist()
                y_vals2 = df12m['Volume Share_client'].tolist()
 
                if len(x_vals2) > 1 and len(y_vals2) > 1:
                    slope2, intercept2 = np.polyfit(x_vals2, y_vals2, 1)
                    y_hat2 = [slope2 * x + intercept2 for x in x_vals2]
                    ss_res2 = sum((y - y_h) ** 2 for y, y_h in zip(y_vals2, y_hat2))
                    ss_tot2 = sum((y - np.mean(y_vals2)) ** 2 for y in y_vals2)
                    r2chart2 = 1 - ss_res2 / ss_tot2 if ss_tot2 != 0 else 0
 
        r_squared_dict[key] = {"chart1": r2chart1, "chart2": r2chart2}
 
        if (r2chart1 is not None and r2chart1 >= 0.5) or (r2chart2 is not None and r2chart2 >= 0.5):
            filtered_modified_price[key] = df
 
    return filtered_modified_price, r_squared_dict

## Summary slide Cleaning 

In [ ]:
def summary(dic, col=" "):
    outputdic = {}

    for k, df in dic.items():
        df = dic[k].copy()
        # df = DetectHeader(df)
        df[df.columns[0]] = df[df.columns[0]].ffill()
        if 'Value Share' not in df.columns or col not in df.columns:
            continue  # Skip if required columns are missing
        df['Value Share'] = pd.to_numeric(df['Value Share'], errors='coerce')
        df = df[df[col] != "Grand Total"]
        df = df[df[df.columns[0]] != "Grand Total"]
        df = df[~df[df.columns[0]].str.contains("Total", na=False)]
        # df = df[df[df.columns[0]].isin(seclis)]
        df[col] = df[col].astype(str)
        df = df[~df[col].str.contains("Other|Others|Other Manufacturer|All Others", na=False, case=False)]
        # if seclis==sectors:
        #      outputdic[f'{categories[0]} | '+k] = df
        # else:
        outputdic[k] = df
    sorted_dic_position = dfSort(outputdic, client_brands, col, num=9, salesCol='Value Share')
    remove_others_from_dic(sorted_dic_position, col)
    dropemptydf(sorted_dic_position)

    return sorted_dic_position
def group_by_key(dic, lis):
    grouped_data = {}
    catdf={}
    for k in dic:
        df=dic[k].copy()
        parts = k.split("|")
        if lis is categories:
            key_part1, key_part2 = parts[1].strip(), parts[0].strip()
        else:
           key_part1, key_part2 = parts[0].strip(), parts[1].strip()

        if key_part2 in lis:
            if key_part1 not in grouped_data:
                grouped_data[key_part1] = []
            grouped_data[key_part1].append(k)
            if lis is categories:
              catdf[key_part2+" | "+key_part1]=df
            else:
                catdf[key_part1+" | "+key_part2]=df
    
    return list(grouped_data.values()),catdf

def indexlist(grouplis):
    final_lis1 = []
    cat_lis = []
    for i in range(len(grouplis)):
        cat_lis += genrateIndexList(grouplis[i], chartIndex=10, chartCount=4)[0]
    final_lis1.append(cat_lis)
    
    return final_lis1 